# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

Now that we have a basic understanding of how SageMaker works we will try to use it to construct a complete project from end to end. Our goal will be to have a simple web page which a user can use to enter a movie review. The web page will then send the review off to our deployed model which will predict the sentiment of the entered review.

## Instructions

Some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this notebook. You will not need to modify the included code beyond what is requested. Sections that begin with '**TODO**' in the header indicate that you need to complete or implement some portion within them. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `# TODO: ...` comment. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions for you to answer which relate to the task and your implementation. Each section where you will answer a question is preceded by a '**Question:**' header. Carefully read each question and provide your answer below the '**Answer:**' header by editing the Markdown cell.

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.

## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

For this project, you will be following the steps in the general outline with some modifications. 

First, you will not be testing the model in its own step. You will still be testing the model, however, you will do it by deploying your model and then using the deployed model by sending the test data to it. One of the reasons for doing this is so that you can make sure that your deployed model is working correctly before moving forward.

In addition, you will deploy and use your trained model a second time. In the second iteration you will customize the way that your trained model is deployed by including some of your own code. In addition, your newly deployed model will be used in the sentiment analysis web app.

## Step 1: Downloading the data

As in the XGBoost in SageMaker notebook, we will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2019-02-12 04:36:08--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  22.6MB/s    in 5.3s    

2019-02-12 04:36:14 (15.2 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

Also, as in the XGBoost notebook, we will be doing some initial data processing. The first few steps are the same as in the XGBoost example. To begin with, we will read in each of the reviews and combine them into a single input structure. Then, we will split the dataset into a training set and a testing set.

In [2]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that we have our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [6]:
print(train_X[100])
print(train_y[100])

This film has special effects which for it's time are very impressive. Some if it is easily explainable with the scenes played backwards but the overlay of moving images on an object on film is surprisingly well done given that this film was made more than 94 years ago.
1


The first step in processing the reviews is to make sure that any html tags that appear should be removed. In addition we wish to tokenize our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis.

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews. As a check to ensure we know how everything is working, try applying `review_to_words` to one of the reviews in the training set.

In [8]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
review_to_words(train_X[100])

['film',
 'special',
 'effect',
 'time',
 'impress',
 'easili',
 'explain',
 'scene',
 'play',
 'backward',
 'overlay',
 'move',
 'imag',
 'object',
 'film',
 'surprisingli',
 'well',
 'done',
 'given',
 'film',
 'made',
 '94',
 'year',
 'ago']

**Question:** Above we mentioned that `review_to_words` method removes html formatting and allows us to tokenize the words found in a review, for example, converting *entertained* and *entertaining* into *entertain* so that they are treated as though they are the same word. What else, if anything, does this method do to the input?

**Answer:**
1. It removes all HTML tags.
2. It also removes all non alphanumerical characters.
3. It also converts all letters to lowercase.
4. It also removes all words found in stopwords
5. It also converts a string into a list of words.

The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results. This is because performing this processing step can take a long time. This way if you are unable to complete the notebook in the current session, you can come back without needing to process the data a second time.

In [9]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [10]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocessed_data.pkl


## Transform the data

In the XGBoost notebook we transformed the data from its word representation to a bag-of-words feature representation. For the model we are going to construct in this notebook we will construct a feature representation which is very similar. To start, we will represent each word as an integer. Of course, some of the words that appear in the reviews occur very infrequently and so likely don't contain much information for the purposes of sentiment analysis. The way we will deal with this problem is that we will fix the size of our working vocabulary and we will only include the words that appear most frequently. We will then combine all of the infrequent words into a single category and, in our case, we will label it as `1`.

Since we will be using a recurrent neural network, it will be convenient if the length of each review is the same. To do this, we will fix a size for our reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.

### (TODO) Create a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.

> **TODO:** Complete the implementation for the `build_dict()` method below. Note that even though the vocab_size is set to `5000`, we only want to construct a mapping for the most frequently appearing `4998` words. This is because we want to reserve the special labels `0` for 'no word' and `1` for 'infrequent word'.

In [16]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    # A dict storing the words that appear in the reviews along with how often they occur
    word_count = {}
    for text in data:
        for w in text:
            if not(w in word_count):
                word_count[w] = 0
            word_count[w] += 1
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    w_freq = []
    for k, v in word_count.items():
        w_freq.append((v, k))
    w_freq.sort(reverse=True)
    
    sorted_words = []
    for _, word in w_freq:
        sorted_words.append(word)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [17]:
word_dict = build_dict(train_X)

**Question:** What are the five most frequently appearing (tokenized) words in the training set? Does it makes sense that these words appear frequently in the training set?

**Answer:**

The words are 'movi', 'film', 'one', 'like', and 'time'.

Yes, because this is about movies and films. And, It makes sense that people can say "this one ...". And, I can see that most of the movie reviews must have been positive given 'like' is in the top 5 frequent words. I think 'time' may appear a lot because people may refer to a point in 'time' in the movie they are reviewing.

In [24]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.
for w, idx in word_dict.items():
    if idx > 1 and idx < 7:
        print('{} th frequent word is {}'.format(idx-1, w))

1 th frequent word is movi
2 th frequent word is film
3 th frequent word is one
4 th frequent word is like
5 th frequent word is time


### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [25]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [26]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Now that we have our word dictionary which allows us to transform the words appearing in the reviews into integers, it is time to make use of it and convert our reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in our case is `500`.

In [27]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [28]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [31]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(train_X_len[300])
train_X[300]

440


array([ 111, 1992,  407, 2876,  221,  445, 1563,  229,  404, 1069,  465,
        261,    1,  285, 1563,    1,  283, 2013,  198,  931,  136,  212,
         61, 1210,  287,  549,   30,  996, 2954,    4, 3284,    1, 1174,
        554,  123,  603, 2939, 1563,   47,  153,  806, 1802, 3284,    1,
       1269, 3056,   51,  202,  247,  364,  216,  175,  264,  239,   62,
        290,  182,    4, 2939,  346,  168,  120,   42,    1,  140,  223,
        361,  279,  188,  797,   38,    3,  361,  279, 1746,   28,  168,
        120,  429,  806,  808,  134,   10,  231,  322, 1689,  168,    2,
        216,  997,    4,   42,  789,   88,  464,  259,   14,  296,  276,
          2,    4,   16,  296,  276,   13,  889,  340,  418,    1, 2202,
        229,    1,    1,   27,  273,   92,    1, 2712,   17, 1004, 1337,
         13,  249,   17,    1,    1,   48,  932,  316,   17, 1226, 1039,
          1, 1168, 4879, 2474, 1168,   17,    1, 2712,  919, 1705,    1,
        100,    1, 1937,    1,    1, 1004, 1337,  2

**Question:** In the cells above we use the `preprocess_data` and `convert_and_pad_data` methods to process both the training and testing set. Why or why not might this be a problem?

**Answer:**

preprocess_data: It is not a problem. In fact, it is good that we are using the same preprocessor for both train and test data for consistency.

convert_and_pad_data: This is also good. We want to make sure that the input is in the expected format and size for both the train and test datasets.

## Step 3: Upload the data to S3

As in the XGBoost notebook, we will need to upload the training dataset to S3 in order for our training code to access it. For now we will save it locally and we will upload to S3 later on.

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [32]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [33]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [34]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**NOTE:** The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. This is fortunate as we will need this later on when we create an endpoint that accepts an arbitrary review. For now, we will just take note of the fact that it resides in the data directory (and so also in the S3 training bucket) and that we will need to make sure it gets saved in the model directory.

## Step 4: Build and Train the PyTorch Model

In the XGBoost notebook we discussed what a model is in the SageMaker framework. In particular, a model comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. In the XGBoost example we used training and inference code that was provided by Amazon. Here we will still be using containers provided by Amazon with the added benefit of being able to include our own custom code.

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [35]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

The important takeaway from the implementation provided is that there are three parameters that we may wish to tweak to improve the performance of our model. These are the embedding dimension, the hidden dimension and the size of the vocabulary. We will likely want to make these parameters configurable in the training script so that if we wish to modify them we do not need to modify the script itself. We will see how to do this later on. To start we will write some of the training code in the notebook so that we can more easily diagnose any issues that arise.

First we will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as we do not have access to a gpu and the compute instance that we are using is not particularly powerful. However, we can work on a small bit of the data to get a feel for how our training script is behaving.

In [36]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### (TODO) Writing the training method

Next we need to write the training code itself. This should be very similar to training methods that you have written before to train PyTorch models. We will leave any difficult aspects such as model saving / loading and parameter loading until a little later.

In [42]:
from torch import nn

def train(model, train_loader, epochs, optimizer, loss_fn, device, clip=5):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            optimizer.zero_grad()
            out = model(batch_X)
            loss = loss_fn(out.squeeze(), batch_y.float())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [44]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 2, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6927168369293213
Epoch: 2, BCELoss: 0.6805244207382202


In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### (TODO) Training the model

When a PyTorch model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained. Inside of the `train` directory is a file called `train.py` which has been provided and which contains most of the necessary code to train our model. The only thing that is missing is the implementation of the `train()` method which you wrote earlier in this notebook.

**TODO**: Copy the `train()` method written above and paste it into the `train/train.py` file where required.

The way that SageMaker passes hyperparameters to the training script is by way of arguments. These arguments can then be parsed and used in the training script. To see how this is done take a look at the provided `train/train.py` file.

In [45]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [46]:
estimator.fit({'training': input_data})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-02-12-05-57-44-386


2019-02-12 05:57:44 Starting - Starting the training job...
2019-02-12 05:57:50 Starting - Launching requested ML instances......
2019-02-12 05:58:48 Starting - Preparing the instances for training......
2019-02-12 06:00:06 Downloading - Downloading input data
2019-02-12 06:00:06 Training - Downloading the training image......
2019-02-12 06:01:07 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-02-12 06:01:08,237 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-02-12 06:01:08,260 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-02-12 06:01:08,265 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-02-12 06:01:08,509 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-02-12 06:01:08,509 sagemak

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Now that we have trained our model, we would like to test it to see how it performs. Currently our model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`. Fortunately for us, SageMaker provides built-in inference code for models with simple inputs such as this.

There is one thing that we need to provide, however, and that is a function which loads the saved model. This function must be called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file which we specified as the entry point. In our case the model loading function has been provided and so no changes need to be made.

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.

**NOTE:** When deploying a model you are asking SageMaker to launch an compute instance that will wait for data to be sent to it. As a result, this compute instance will continue to run until *you* shut it down. This is important to know since the cost of a deployed endpoint depends on how long it has been running for.

In other words **If you are no longer using a deployed endpoint, shut it down!**

**TODO:** Deploy the trained model.

In [47]:
# TODO: Deploy the trained model
predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-02-12-05-57-44-386
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-02-12-05-57-44-386


--------------------------------------------------------------------------!

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [48]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [49]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [50]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.83432

**Question:** How does this model compare to the XGBoost model you created earlier? Why might these two models perform differently on this dataset? Which do *you* think is better for sentiment analysis?

**Answer:**
XGBoost model's accuracy on the test dataset is 0.8566. This one's accuracy is 0.83432. I think there isn't too much of a difference, but if I have to choose, XGBoost model seems to perform better by 2%.

### (TODO) More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [52]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order process the review we will need to repeat these two steps.

**TODO**: Using the `review_to_words` and `convert_and_pad` methods from section one, convert `test_review` into a numpy array `test_data` suitable to send to our model. Remember that our model expects input of the form `review_length, review[500]`.

In [88]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data = review_to_words(test_review)
test_data, test_data_len = convert_and_pad_data(word_dict, [test_data])
test_data = pd.concat([pd.DataFrame(test_data_len), pd.DataFrame(test_data)], axis=1)
test_data = test_data.values

Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [89]:
predictor.predict(test_data).item()

0.8736770153045654

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Of course, just like in the XGBoost notebook, once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [90]:
estimator.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: sagemaker-pytorch-2019-02-12-05-57-44-386


## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

As we saw above, by default the estimator which we created, when deployed, will use the entry script and directory which we provided when creating the model. However, since we now wish to accept a string as input and our model expects a processed review, we need to write some custom inference code.

We will store the code that we write in the `serve` directory. Provided in this directory is the `model.py` file that we used to construct our model, a `utils.py` file which contains the `review_to_words` and `convert_and_pad` pre-processing functions which we used during the initial data processing, and `predict.py`, the file which will contain our custom inference code. Note also that `requirements.txt` is present which will tell SageMaker what Python libraries are required by our custom inference code.

When deploying a PyTorch model in SageMaker, you are expected to provide four functions which the SageMaker inference container will use.
 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.

For the simple website that we are constructing during this project, the `input_fn` and `output_fn` methods are relatively straightforward. We only require being able to accept a string as input and we expect to return a single value as output. You might imagine though that in a more complex application the input or output may be image data or some other binary data which would require some effort to serialize.

### (TODO) Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [91]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

In [92]:
print(word_dict)

{'movi': 2, 'film': 3, 'one': 4, 'like': 5, 'time': 6, 'good': 7, 'make': 8, 'charact': 9, 'get': 10, 'see': 11, 'watch': 12, 'stori': 13, 'even': 14, 'would': 15, 'realli': 16, 'well': 17, 'scene': 18, 'look': 19, 'show': 20, 'much': 21, 'end': 22, 'peopl': 23, 'bad': 24, 'go': 25, 'great': 26, 'also': 27, 'first': 28, 'love': 29, 'think': 30, 'way': 31, 'act': 32, 'play': 33, 'made': 34, 'thing': 35, 'could': 36, 'know': 37, 'say': 38, 'seem': 39, 'work': 40, 'plot': 41, 'two': 42, 'actor': 43, 'year': 44, 'come': 45, 'mani': 46, 'seen': 47, 'take': 48, 'want': 49, 'life': 50, 'never': 51, 'littl': 52, 'best': 53, 'tri': 54, 'man': 55, 'ever': 56, 'give': 57, 'better': 58, 'still': 59, 'perform': 60, 'find': 61, 'feel': 62, 'part': 63, 'back': 64, 'use': 65, 'someth': 66, 'director': 67, 'actual': 68, 'interest': 69, 'lot': 70, 'real': 71, 'old': 72, 'cast': 73, 'though': 74, 'live': 75, 'star': 76, 'enjoy': 77, 'guy': 78, 'anoth': 79, 'new': 80, 'role': 81, 'noth': 82, '10': 83, 'fu

As mentioned earlier, the `model_fn` method is the same as the one provided in the training code and the `input_fn` and `output_fn` methods are very simple and your task will be to complete the `predict_fn` method. Make sure that you save the completed file as `predict.py` in the `serve` directory.

**TODO**: Complete the `predict_fn()` method in the `serve/predict.py` file.

In [153]:
import argparse
import json
import os
import pickle
import sys
# import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

# from model import LSTMClassifier

# from utils import review_to_words, convert_and_pad
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

import pickle

import os
import glob

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def predict_fn(input_data, model):
    print('Inferring sentiment of input data.')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if model.word_dict is None:
        raise Exception('Model has not been loaded properly, no word_dict.')
    
    # TODO: Process input_data so that it is ready to be sent to our model.
    #       You should produce two variables:
    #         data_X   - A sequence of length 500 which represents the converted review
    #         data_len - The length of the review

    data_X = review_to_words(input_data)
    data_X, data_len = convert_and_pad(model.word_dict, data_X)

    # Using data_X and data_len we construct an appropriate input tensor. Remember
    # that our model expects input data of the form 'len, review[500]'.
    data_pack = np.hstack((data_len, data_X))
    data_pack = data_pack.reshape(1, -1)
    
    data = torch.from_numpy(data_pack)
    data = data.to(device)

    # Make sure to put the model into evaluation mode
    model.eval()

    # TODO: Compute the result of applying the model to the input data. The variable `result` should
    #       be a numpy array which contains a single integer which is either 1 or 0
    prediction = model(data)
    prediction = torch.round(prediction).cpu().squeeze().int()
    result = prediction.detach().numpy()

    return result


In [154]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
net = LSTMClassifier(32, 100, 5000).to(device)
net.word_dict = {'movi': 2, 'film': 3, 'one': 4, 'like': 5, 'time': 6, 'good': 7, 'make': 8, 'charact': 9, 'get': 10, 'see': 11, 'watch': 12, 'stori': 13, 'even': 14, 'would': 15, 'realli': 16, 'well': 17, 'scene': 18, 'look': 19, 'show': 20, 'much': 21, 'end': 22, 'peopl': 23, 'bad': 24, 'go': 25, 'great': 26, 'also': 27, 'first': 28, 'love': 29, 'think': 30, 'way': 31, 'act': 32, 'play': 33, 'made': 34, 'thing': 35, 'could': 36, 'know': 37, 'say': 38, 'seem': 39, 'work': 40, 'plot': 41, 'two': 42, 'actor': 43, 'year': 44, 'come': 45, 'mani': 46, 'seen': 47, 'take': 48, 'want': 49, 'life': 50, 'never': 51, 'littl': 52, 'best': 53, 'tri': 54, 'man': 55, 'ever': 56, 'give': 57, 'better': 58, 'still': 59, 'perform': 60, 'find': 61, 'feel': 62, 'part': 63, 'back': 64, 'use': 65, 'someth': 66, 'director': 67, 'actual': 68, 'interest': 69, 'lot': 70, 'real': 71, 'old': 72, 'cast': 73, 'though': 74, 'live': 75, 'star': 76, 'enjoy': 77, 'guy': 78, 'anoth': 79, 'new': 80, 'role': 81, 'noth': 82, '10': 83, 'funni': 84, 'music': 85, 'point': 86, 'start': 87, 'set': 88, 'girl': 89, 'origin': 90, 'day': 91, 'world': 92, 'everi': 93, 'believ': 94, 'turn': 95, 'quit': 96, 'us': 97, 'direct': 98, 'thought': 99, 'fact': 100, 'minut': 101, 'horror': 102, 'kill': 103, 'action': 104, 'comedi': 105, 'pretti': 106, 'young': 107, 'wonder': 108, 'happen': 109, 'around': 110, 'got': 111, 'effect': 112, 'right': 113, 'long': 114, 'howev': 115, 'big': 116, 'line': 117, 'famili': 118, 'enough': 119, 'seri': 120, 'may': 121, 'need': 122, 'fan': 123, 'bit': 124, 'script': 125, 'beauti': 126, 'person': 127, 'becom': 128, 'without': 129, 'must': 130, 'alway': 131, 'friend': 132, 'tell': 133, 'reason': 134, 'saw': 135, 'last': 136, 'final': 137, 'kid': 138, 'almost': 139, 'put': 140, 'least': 141, 'sure': 142, 'done': 143, 'whole': 144, 'place': 145, 'complet': 146, 'kind': 147, 'expect': 148, 'differ': 149, 'shot': 150, 'far': 151, 'mean': 152, 'anyth': 153, 'book': 154, 'laugh': 155, 'might': 156, 'name': 157, 'sinc': 158, 'begin': 159, '2': 160, 'probabl': 161, 'woman': 162, 'help': 163, 'entertain': 164, 'let': 165, 'screen': 166, 'call': 167, 'tv': 168, 'moment': 169, 'away': 170, 'read': 171, 'yet': 172, 'rather': 173, 'worst': 174, 'run': 175, 'fun': 176, 'lead': 177, 'hard': 178, 'audienc': 179, 'idea': 180, 'anyon': 181, 'episod': 182, 'american': 183, 'found': 184, 'appear': 185, 'bore': 186, 'especi': 187, 'although': 188, 'hope': 189, 'keep': 190, 'cours': 191, 'anim': 192, 'job': 193, 'goe': 194, 'move': 195, 'sens': 196, 'version': 197, 'dvd': 198, 'war': 199, 'money': 200, 'someon': 201, 'mind': 202, 'mayb': 203, 'problem': 204, 'true': 205, 'hous': 206, 'everyth': 207, 'nice': 208, 'second': 209, 'rate': 210, 'three': 211, 'night': 212, 'follow': 213, 'face': 214, 'recommend': 215, 'product': 216, 'main': 217, 'worth': 218, 'leav': 219, 'human': 220, 'special': 221, 'excel': 222, 'togeth': 223, 'wast': 224, 'sound': 225, 'everyon': 226, 'john': 227, 'hand': 228, '1': 229, 'father': 230, 'later': 231, 'eye': 232, 'said': 233, 'view': 234, 'instead': 235, 'review': 236, 'boy': 237, 'high': 238, 'hour': 239, 'miss': 240, 'talk': 241, 'classic': 242, 'wife': 243, 'understand': 244, 'left': 245, 'care': 246, 'black': 247, 'death': 248, 'open': 249, 'murder': 250, 'write': 251, 'half': 252, 'head': 253, 'rememb': 254, 'chang': 255, 'viewer': 256, 'fight': 257, 'gener': 258, 'surpris': 259, 'short': 260, 'includ': 261, 'die': 262, 'fall': 263, 'less': 264, 'els': 265, 'entir': 266, 'piec': 267, 'involv': 268, 'pictur': 269, 'simpli': 270, 'top': 271, 'power': 272, 'home': 273, 'total': 274, 'usual': 275, 'budget': 276, 'attempt': 277, 'suppos': 278, 'releas': 279, 'hollywood': 280, 'terribl': 281, 'song': 282, 'men': 283, 'possibl': 284, 'featur': 285, 'portray': 286, 'disappoint': 287, 'poor': 288, '3': 289, 'coupl': 290, 'stupid': 291, 'camera': 292, 'dead': 293, 'wrong': 294, 'produc': 295, 'low': 296, 'video': 297, 'either': 298, 'aw': 299, 'definit': 300, 'except': 301, 'rest': 302, 'given': 303, 'absolut': 304, 'women': 305, 'lack': 306, 'word': 307, 'writer': 308, 'titl': 309, 'talent': 310, 'decid': 311, 'full': 312, 'perfect': 313, 'along': 314, 'style': 315, 'close': 316, 'truli': 317, 'school': 318, 'save': 319, 'emot': 320, 'sex': 321, 'age': 322, 'next': 323, 'bring': 324, 'mr': 325, 'case': 326, 'killer': 327, 'heart': 328, 'comment': 329, 'sort': 330, 'creat': 331, 'perhap': 332, 'came': 333, 'brother': 334, 'sever': 335, 'joke': 336, 'art': 337, 'dialogu': 338, 'game': 339, 'small': 340, 'base': 341, 'flick': 342, 'written': 343, 'sequenc': 344, 'meet': 345, 'earli': 346, 'often': 347, 'other': 348, 'mother': 349, 'develop': 350, 'humor': 351, 'actress': 352, 'consid': 353, 'dark': 354, 'guess': 355, 'amaz': 356, 'unfortun': 357, 'lost': 358, 'light': 359, 'exampl': 360, 'cinema': 361, 'drama': 362, 'ye': 363, 'white': 364, 'experi': 365, 'imagin': 366, 'mention': 367, 'stop': 368, 'natur': 369, 'forc': 370, 'manag': 371, 'felt': 372, 'present': 373, 'cut': 374, 'children': 375, 'fail': 376, 'son': 377, 'support': 378, 'qualiti': 379, 'car': 380, 'ask': 381, 'hit': 382, 'side': 383, 'voic': 384, 'extrem': 385, 'impress': 386, 'wors': 387, 'evil': 388, 'went': 389, 'stand': 390, 'certainli': 391, 'basic': 392, 'oh': 393, 'overal': 394, 'favorit': 395, 'horribl': 396, 'mysteri': 397, 'number': 398, 'type': 399, 'danc': 400, 'wait': 401, 'hero': 402, 'alreadi': 403, '5': 404, 'learn': 405, 'matter': 406, '4': 407, 'michael': 408, 'genr': 409, 'fine': 410, 'despit': 411, 'throughout': 412, 'walk': 413, 'success': 414, 'histori': 415, 'question': 416, 'zombi': 417, 'town': 418, 'relationship': 419, 'realiz': 420, 'past': 421, 'child': 422, 'daughter': 423, 'late': 424, 'b': 425, 'wish': 426, 'hate': 427, 'credit': 428, 'event': 429, 'theme': 430, 'touch': 431, 'citi': 432, 'today': 433, 'sometim': 434, 'behind': 435, 'god': 436, 'twist': 437, 'sit': 438, 'stay': 439, 'deal': 440, 'annoy': 441, 'abl': 442, 'rent': 443, 'pleas': 444, 'edit': 445, 'blood': 446, 'deserv': 447, 'comic': 448, 'anyway': 449, 'appar': 450, 'soon': 451, 'gave': 452, 'etc': 453, 'level': 454, 'slow': 455, 'chanc': 456, 'score': 457, 'bodi': 458, 'brilliant': 459, 'incred': 460, 'figur': 461, 'situat': 462, 'self': 463, 'major': 464, 'stuff': 465, 'decent': 466, 'element': 467, 'return': 468, 'dream': 469, 'obvious': 470, 'order': 471, 'continu': 472, 'pace': 473, 'ridicul': 474, 'happi': 475, 'highli': 476, 'group': 477, 'add': 478, 'thank': 479, 'ladi': 480, 'novel': 481, 'speak': 482, 'pain': 483, 'career': 484, 'shoot': 485, 'strang': 486, 'heard': 487, 'sad': 488, 'polic': 489, 'husband': 490, 'import': 491, 'break': 492, 'took': 493, 'strong': 494, 'cannot': 495, 'robert': 496, 'predict': 497, 'violenc': 498, 'hilari': 499, 'recent': 500, 'countri': 501, 'known': 502, 'particularli': 503, 'pick': 504, 'documentari': 505, 'season': 506, 'critic': 507, 'jame': 508, 'compar': 509, 'obviou': 510, 'alon': 511, 'told': 512, 'state': 513, 'visual': 514, 'rock': 515, 'theater': 516, 'offer': 517, 'exist': 518, 'opinion': 519, 'gore': 520, 'hold': 521, 'crap': 522, 'result': 523, 'room': 524, 'realiti': 525, 'hear': 526, 'effort': 527, 'clich': 528, 'thriller': 529, 'caus': 530, 'serious': 531, 'sequel': 532, 'explain': 533, 'king': 534, 'local': 535, 'ago': 536, 'none': 537, 'hell': 538, 'note': 539, 'allow': 540, 'sister': 541, 'david': 542, 'simpl': 543, 'femal': 544, 'deliv': 545, 'ok': 546, 'convinc': 547, 'class': 548, 'check': 549, 'suspens': 550, 'win': 551, 'oscar': 552, 'buy': 553, 'huge': 554, 'valu': 555, 'sexual': 556, 'scari': 557, 'cool': 558, 'similar': 559, 'excit': 560, 'provid': 561, 'exactli': 562, 'apart': 563, 'shown': 564, 'avoid': 565, 'seriou': 566, 'english': 567, 'whose': 568, 'taken': 569, 'cinematographi': 570, 'shock': 571, 'polit': 572, 'spoiler': 573, 'offic': 574, 'across': 575, 'middl': 576, 'street': 577, 'pass': 578, 'messag': 579, 'somewhat': 580, 'silli': 581, 'charm': 582, 'modern': 583, 'filmmak': 584, 'confus': 585, 'form': 586, 'tale': 587, 'singl': 588, 'jack': 589, 'mostli': 590, 'william': 591, 'carri': 592, 'attent': 593, 'sing': 594, 'subject': 595, 'five': 596, 'richard': 597, 'prove': 598, 'team': 599, 'stage': 600, 'unlik': 601, 'cop': 602, 'georg': 603, 'televis': 604, 'monster': 605, 'earth': 606, 'villain': 607, 'cover': 608, 'pay': 609, 'marri': 610, 'toward': 611, 'build': 612, 'pull': 613, 'parent': 614, 'due': 615, 'respect': 616, 'fill': 617, 'four': 618, 'dialog': 619, 'remind': 620, 'futur': 621, 'weak': 622, 'typic': 623, '7': 624, 'cheap': 625, 'intellig': 626, 'british': 627, 'atmospher': 628, 'clearli': 629, '80': 630, 'paul': 631, 'non': 632, 'dog': 633, 'knew': 634, 'fast': 635, 'artist': 636, '8': 637, 'crime': 638, 'easili': 639, 'escap': 640, 'doubt': 641, 'adult': 642, 'detail': 643, 'date': 644, 'romant': 645, 'member': 646, 'fire': 647, 'gun': 648, 'drive': 649, 'straight': 650, 'fit': 651, 'beyond': 652, 'attack': 653, 'imag': 654, 'upon': 655, 'posit': 656, 'whether': 657, 'peter': 658, 'fantast': 659, 'captur': 660, 'aspect': 661, 'appreci': 662, 'ten': 663, 'plan': 664, 'discov': 665, 'remain': 666, 'period': 667, 'near': 668, 'realist': 669, 'air': 670, 'mark': 671, 'red': 672, 'dull': 673, 'adapt': 674, 'within': 675, 'spend': 676, 'lose': 677, 'materi': 678, 'color': 679, 'chase': 680, 'mari': 681, 'storylin': 682, 'forget': 683, 'bunch': 684, 'clear': 685, 'lee': 686, 'victim': 687, 'nearli': 688, 'box': 689, 'york': 690, 'match': 691, 'inspir': 692, 'mess': 693, 'finish': 694, 'standard': 695, 'easi': 696, 'truth': 697, 'suffer': 698, 'busi': 699, 'space': 700, 'dramat': 701, 'bill': 702, 'western': 703, 'e': 704, 'list': 705, 'battl': 706, 'notic': 707, 'de': 708, 'french': 709, 'ad': 710, '9': 711, 'tom': 712, 'larg': 713, 'among': 714, 'eventu': 715, 'train': 716, 'accept': 717, 'agre': 718, 'spirit': 719, 'soundtrack': 720, 'third': 721, 'teenag': 722, 'soldier': 723, 'adventur': 724, 'suggest': 725, 'sorri': 726, 'famou': 727, 'drug': 728, 'normal': 729, 'cri': 730, 'babi': 731, 'ultim': 732, 'troubl': 733, 'contain': 734, 'certain': 735, 'cultur': 736, 'romanc': 737, 'rare': 738, 'lame': 739, 'somehow': 740, 'mix': 741, 'disney': 742, 'gone': 743, 'cartoon': 744, 'student': 745, 'reveal': 746, 'fear': 747, 'suck': 748, 'kept': 749, 'attract': 750, 'appeal': 751, 'premis': 752, 'secret': 753, 'greatest': 754, 'design': 755, 'shame': 756, 'throw': 757, 'scare': 758, 'copi': 759, 'wit': 760, 'america': 761, 'admit': 762, 'relat': 763, 'particular': 764, 'brought': 765, 'screenplay': 766, 'whatev': 767, 'pure': 768, '70': 769, 'harri': 770, 'averag': 771, 'master': 772, 'describ': 773, 'treat': 774, 'male': 775, '20': 776, 'issu': 777, 'fantasi': 778, 'warn': 779, 'inde': 780, 'forward': 781, 'background': 782, 'project': 783, 'free': 784, 'memor': 785, 'japanes': 786, 'poorli': 787, 'award': 788, 'locat': 789, 'potenti': 790, 'amus': 791, 'struggl': 792, 'weird': 793, 'magic': 794, 'societi': 795, 'okay': 796, 'imdb': 797, 'doctor': 798, 'accent': 799, 'water': 800, 'hot': 801, 'express': 802, 'dr': 803, 'alien': 804, '30': 805, 'odd': 806, 'crazi': 807, 'choic': 808, 'studio': 809, 'fiction': 810, 'control': 811, 'becam': 812, 'masterpiec': 813, 'fli': 814, 'difficult': 815, 'joe': 816, 'scream': 817, 'costum': 818, 'lover': 819, 'uniqu': 820, 'refer': 821, 'remak': 822, 'vampir': 823, 'girlfriend': 824, 'prison': 825, 'execut': 826, 'wear': 827, 'jump': 828, 'wood': 829, 'unless': 830, 'creepi': 831, 'cheesi': 832, 'superb': 833, 'otherwis': 834, 'parti': 835, 'roll': 836, 'ghost': 837, 'public': 838, 'mad': 839, 'depict': 840, 'week': 841, 'moral': 842, 'jane': 843, 'earlier': 844, 'badli': 845, 'fi': 846, 'dumb': 847, 'grow': 848, 'flaw': 849, 'sci': 850, 'deep': 851, 'maker': 852, 'cat': 853, 'older': 854, 'footag': 855, 'connect': 856, 'plenti': 857, 'bother': 858, 'outsid': 859, 'stick': 860, 'gay': 861, 'catch': 862, 'plu': 863, 'co': 864, 'popular': 865, 'equal': 866, 'social': 867, 'quickli': 868, 'disturb': 869, 'perfectli': 870, 'dress': 871, 'era': 872, '90': 873, 'mistak': 874, 'lie': 875, 'ride': 876, 'previou': 877, 'combin': 878, 'concept': 879, 'band': 880, 'surviv': 881, 'rich': 882, 'answer': 883, 'front': 884, 'sweet': 885, 'christma': 886, 'insid': 887, 'eat': 888, 'concern': 889, 'bare': 890, 'listen': 891, 'ben': 892, 'beat': 893, 'c': 894, 'term': 895, 'serv': 896, 'meant': 897, 'la': 898, 'german': 899, 'stereotyp': 900, 'hardli': 901, 'law': 902, 'innoc': 903, 'desper': 904, 'promis': 905, 'memori': 906, 'intent': 907, 'cute': 908, 'variou': 909, 'steal': 910, 'inform': 911, 'brain': 912, 'post': 913, 'tone': 914, 'island': 915, 'amount': 916, 'track': 917, 'nuditi': 918, 'compani': 919, 'store': 920, 'claim': 921, 'hair': 922, 'flat': 923, '50': 924, 'univers': 925, 'land': 926, 'scott': 927, 'kick': 928, 'fairli': 929, 'danger': 930, 'player': 931, 'step': 932, 'plain': 933, 'crew': 934, 'toni': 935, 'share': 936, 'tast': 937, 'centuri': 938, 'engag': 939, 'achiev': 940, 'travel': 941, 'cold': 942, 'suit': 943, 'rip': 944, 'record': 945, 'sadli': 946, 'manner': 947, 'wrote': 948, 'tension': 949, 'spot': 950, 'intens': 951, 'fascin': 952, 'familiar': 953, 'remark': 954, 'depth': 955, 'burn': 956, 'histor': 957, 'destroy': 958, 'sleep': 959, 'purpos': 960, 'languag': 961, 'ruin': 962, 'ignor': 963, 'delight': 964, 'unbeliev': 965, 'italian': 966, 'soul': 967, 'collect': 968, 'abil': 969, 'detect': 970, 'clever': 971, 'violent': 972, 'rape': 973, 'reach': 974, 'door': 975, 'trash': 976, 'scienc': 977, 'liter': 978, 'reveng': 979, 'commun': 980, 'caught': 981, 'creatur': 982, 'trip': 983, 'approach': 984, 'intrigu': 985, 'fashion': 986, 'skill': 987, 'paint': 988, 'introduc': 989, 'complex': 990, 'channel': 991, 'camp': 992, 'christian': 993, 'hole': 994, 'extra': 995, 'mental': 996, 'limit': 997, 'immedi': 998, 'ann': 999, 'slightli': 1000, 'million': 1001, 'mere': 1002, 'comput': 1003, '6': 1004, 'slasher': 1005, 'conclus': 1006, 'suddenli': 1007, 'imposs': 1008, 'teen': 1009, 'neither': 1010, 'crimin': 1011, 'spent': 1012, 'physic': 1013, 'nation': 1014, 'respons': 1015, 'planet': 1016, 'receiv': 1017, 'fake': 1018, 'sick': 1019, 'blue': 1020, 'bizarr': 1021, 'embarrass': 1022, 'indian': 1023, 'ring': 1024, '15': 1025, 'pop': 1026, 'drop': 1027, 'drag': 1028, 'haunt': 1029, 'suspect': 1030, 'pointless': 1031, 'search': 1032, 'edg': 1033, 'handl': 1034, 'common': 1035, 'biggest': 1036, 'hurt': 1037, 'faith': 1038, 'arriv': 1039, 'technic': 1040, 'angel': 1041, 'genuin': 1042, 'dad': 1043, 'solid': 1044, 'f': 1045, 'awesom': 1046, 'van': 1047, 'former': 1048, 'focu': 1049, 'colleg': 1050, 'count': 1051, 'tear': 1052, 'heavi': 1053, 'wall': 1054, 'rais': 1055, 'younger': 1056, 'visit': 1057, 'laughabl': 1058, 'sign': 1059, 'fair': 1060, 'excus': 1061, 'cult': 1062, 'tough': 1063, 'motion': 1064, 'key': 1065, 'super': 1066, 'desir': 1067, 'stun': 1068, 'addit': 1069, 'exploit': 1070, 'cloth': 1071, 'tortur': 1072, 'smith': 1073, 'race': 1074, 'davi': 1075, 'cross': 1076, 'author': 1077, 'jim': 1078, 'minor': 1079, 'focus': 1080, 'consist': 1081, 'compel': 1082, 'pathet': 1083, 'commit': 1084, 'chemistri': 1085, 'park': 1086, 'tradit': 1087, 'obsess': 1088, 'frank': 1089, 'grade': 1090, 'asid': 1091, '60': 1092, 'brutal': 1093, 'steve': 1094, 'somewher': 1095, 'u': 1096, 'rule': 1097, 'opportun': 1098, 'grant': 1099, 'explor': 1100, 'depress': 1101, 'honest': 1102, 'besid': 1103, 'dub': 1104, 'anti': 1105, 'trailer': 1106, 'intend': 1107, 'bar': 1108, 'west': 1109, 'scientist': 1110, 'regard': 1111, 'longer': 1112, 'judg': 1113, 'decad': 1114, 'silent': 1115, 'creativ': 1116, 'armi': 1117, 'wild': 1118, 'stewart': 1119, 'south': 1120, 'g': 1121, 'draw': 1122, 'road': 1123, 'govern': 1124, 'ex': 1125, 'boss': 1126, 'practic': 1127, 'surprisingli': 1128, 'motiv': 1129, 'gang': 1130, 'festiv': 1131, 'club': 1132, 'redeem': 1133, 'page': 1134, 'london': 1135, 'green': 1136, 'militari': 1137, 'machin': 1138, 'idiot': 1139, 'display': 1140, 'aliv': 1141, 'thrill': 1142, 'repeat': 1143, 'yeah': 1144, 'nobodi': 1145, 'folk': 1146, '100': 1147, '40': 1148, 'journey': 1149, 'garbag': 1150, 'tire': 1151, 'smile': 1152, 'ground': 1153, 'mood': 1154, 'bought': 1155, 'stone': 1156, 'sam': 1157, 'cost': 1158, 'noir': 1159, 'mouth': 1160, 'terrif': 1161, 'agent': 1162, 'utterli': 1163, 'requir': 1164, 'sexi': 1165, 'honestli': 1166, 'area': 1167, 'report': 1168, 'geniu': 1169, 'investig': 1170, 'humour': 1171, 'glad': 1172, 'enter': 1173, 'serial': 1174, 'passion': 1175, 'occasion': 1176, 'narr': 1177, 'marriag': 1178, 'climax': 1179, 'studi': 1180, 'industri': 1181, 'ship': 1182, 'nowher': 1183, 'demon': 1184, 'charli': 1185, 'center': 1186, 'loos': 1187, 'hors': 1188, 'bear': 1189, 'wow': 1190, 'hang': 1191, 'graphic': 1192, 'giant': 1193, 'admir': 1194, 'send': 1195, 'loud': 1196, 'damn': 1197, 'subtl': 1198, 'rel': 1199, 'profession': 1200, 'nake': 1201, 'blow': 1202, 'bottom': 1203, 'insult': 1204, 'batman': 1205, 'r': 1206, 'kelli': 1207, 'doubl': 1208, 'boyfriend': 1209, 'initi': 1210, 'frame': 1211, 'opera': 1212, 'gem': 1213, 'drawn': 1214, 'cinemat': 1215, 'church': 1216, 'challeng': 1217, 'affect': 1218, 'seek': 1219, 'nightmar': 1220, 'l': 1221, 'j': 1222, 'fulli': 1223, 'evid': 1224, 'essenti': 1225, 'conflict': 1226, 'arm': 1227, 'wind': 1228, 'henri': 1229, 'grace': 1230, 'christoph': 1231, 'witch': 1232, 'narrat': 1233, 'assum': 1234, 'push': 1235, 'hunt': 1236, 'wise': 1237, 'chri': 1238, 'repres': 1239, 'nomin': 1240, 'month': 1241, 'sceneri': 1242, 'hide': 1243, 'avail': 1244, 'affair': 1245, 'thu': 1246, 'smart': 1247, 'justic': 1248, 'bond': 1249, 'outstand': 1250, 'interview': 1251, 'flashback': 1252, 'satisfi': 1253, 'presenc': 1254, 'constantli': 1255, 'central': 1256, 'bed': 1257, 'sell': 1258, 'iron': 1259, 'content': 1260, 'gag': 1261, 'everybodi': 1262, 'slowli': 1263, 'hotel': 1264, 'hire': 1265, 'system': 1266, 'thrown': 1267, 'individu': 1268, 'hey': 1269, 'charl': 1270, 'adam': 1271, 'mediocr': 1272, 'jone': 1273, 'allen': 1274, 'ray': 1275, 'lesson': 1276, 'billi': 1277, 'photographi': 1278, 'cameo': 1279, 'pari': 1280, 'fellow': 1281, 'strike': 1282, 'rise': 1283, 'independ': 1284, 'brief': 1285, 'absurd': 1286, 'neg': 1287, 'phone': 1288, 'impact': 1289, 'model': 1290, 'ill': 1291, 'born': 1292, 'spoil': 1293, 'fresh': 1294, 'angl': 1295, 'likabl': 1296, 'abus': 1297, 'hill': 1298, 'discuss': 1299, 'sight': 1300, 'ahead': 1301, 'sent': 1302, 'photograph': 1303, 'shine': 1304, 'occur': 1305, 'logic': 1306, 'blame': 1307, 'mainli': 1308, 'bruce': 1309, 'skip': 1310, 'forev': 1311, 'commerci': 1312, 'teacher': 1313, 'surround': 1314, 'segment': 1315, 'held': 1316, 'zero': 1317, 'blond': 1318, 'trap': 1319, 'summer': 1320, 'satir': 1321, 'resembl': 1322, 'six': 1323, 'queen': 1324, 'fool': 1325, 'ball': 1326, 'twice': 1327, 'tragedi': 1328, 'sub': 1329, 'reaction': 1330, 'pack': 1331, 'bomb': 1332, 'will': 1333, 'protagonist': 1334, 'hospit': 1335, 'sport': 1336, 'mile': 1337, 'vote': 1338, 'trust': 1339, 'mom': 1340, 'jerri': 1341, 'drink': 1342, 'encount': 1343, 'plane': 1344, 'station': 1345, 'program': 1346, 'current': 1347, 'al': 1348, 'martin': 1349, 'choos': 1350, 'celebr': 1351, 'join': 1352, 'tragic': 1353, 'round': 1354, 'lord': 1355, 'field': 1356, 'favourit': 1357, 'vision': 1358, 'robot': 1359, 'jean': 1360, 'tie': 1361, 'arthur': 1362, 'roger': 1363, 'random': 1364, 'fortun': 1365, 'psycholog': 1366, 'intern': 1367, 'dread': 1368, 'prefer': 1369, 'nonsens': 1370, 'improv': 1371, 'epic': 1372, 'pleasur': 1373, 'legend': 1374, 'highlight': 1375, 'formula': 1376, 'tape': 1377, 'dollar': 1378, '11': 1379, 'wide': 1380, 'thin': 1381, 'porn': 1382, 'object': 1383, 'gorgeou': 1384, 'fox': 1385, 'ugli': 1386, 'influenc': 1387, 'buddi': 1388, 'prepar': 1389, 'nasti': 1390, 'ii': 1391, 'warm': 1392, 'supposedli': 1393, 'reflect': 1394, 'progress': 1395, 'youth': 1396, 'worthi': 1397, 'unusu': 1398, 'length': 1399, 'latter': 1400, 'crash': 1401, 'superior': 1402, 'shop': 1403, 'seven': 1404, 'childhood': 1405, 'theatr': 1406, 'remot': 1407, 'pilot': 1408, 'paid': 1409, 'funniest': 1410, 'disgust': 1411, 'trick': 1412, 'fell': 1413, 'convers': 1414, 'castl': 1415, 'rob': 1416, 'gangster': 1417, 'establish': 1418, 'disast': 1419, 'suicid': 1420, 'mine': 1421, 'ident': 1422, 'heaven': 1423, 'disappear': 1424, 'tend': 1425, 'singer': 1426, 'mask': 1427, 'heroin': 1428, 'forgotten': 1429, 'decis': 1430, 'partner': 1431, 'brian': 1432, 'recogn': 1433, 'desert': 1434, 'alan': 1435, 'thoroughli': 1436, 'stuck': 1437, 'sky': 1438, 'p': 1439, 'ms': 1440, 'replac': 1441, 'accur': 1442, 'market': 1443, 'uncl': 1444, 'seemingli': 1445, 'eddi': 1446, 'danni': 1447, 'commentari': 1448, 'clue': 1449, 'andi': 1450, 'jackson': 1451, 'devil': 1452, 'therefor': 1453, 'that': 1454, 'refus': 1455, 'pair': 1456, 'unit': 1457, 'river': 1458, 'fault': 1459, 'fate': 1460, 'ed': 1461, 'accid': 1462, 'tune': 1463, 'afraid': 1464, 'stephen': 1465, 'russian': 1466, 'hidden': 1467, 'clean': 1468, 'test': 1469, 'readi': 1470, 'quick': 1471, 'irrit': 1472, 'instanc': 1473, 'convey': 1474, 'captain': 1475, 'european': 1476, 'insan': 1477, 'frustrat': 1478, 'daniel': 1479, 'wed': 1480, 'rescu': 1481, 'food': 1482, 'chines': 1483, '1950': 1484, 'lock': 1485, 'dirti': 1486, 'angri': 1487, 'joy': 1488, 'steven': 1489, 'price': 1490, 'cage': 1491, 'bland': 1492, 'rang': 1493, 'anymor': 1494, 'wooden': 1495, 'rush': 1496, 'news': 1497, 'n': 1498, 'jason': 1499, 'worri': 1500, 'twenti': 1501, 'martial': 1502, 'led': 1503, 'board': 1504, '12': 1505, 'transform': 1506, 'symbol': 1507, 'hunter': 1508, 'cgi': 1509, 'x': 1510, 'sentiment': 1511, 'piti': 1512, 'onto': 1513, 'johnni': 1514, 'invent': 1515, 'process': 1516, 'explan': 1517, 'attitud': 1518, 'owner': 1519, 'awar': 1520, 'aim': 1521, 'target': 1522, 'necessari': 1523, 'floor': 1524, 'favor': 1525, 'energi': 1526, 'religi': 1527, 'opposit': 1528, 'window': 1529, 'insight': 1530, 'chick': 1531, 'blind': 1532, 'movement': 1533, 'research': 1534, 'possess': 1535, 'mountain': 1536, 'deepli': 1537, 'comparison': 1538, 'whatsoev': 1539, 'rain': 1540, 'grand': 1541, 'comed': 1542, 'shadow': 1543, 'mid': 1544, 'began': 1545, 'bank': 1546, 'princ': 1547, 'parodi': 1548, 'weapon': 1549, 'taylor': 1550, 'pre': 1551, 'friendship': 1552, 'credibl': 1553, 'teach': 1554, 'flesh': 1555, 'dougla': 1556, 'terror': 1557, 'protect': 1558, 'hint': 1559, 'bloodi': 1560, 'marvel': 1561, 'watchabl': 1562, 'superman': 1563, 'load': 1564, 'leader': 1565, 'drunk': 1566, 'anybodi': 1567, 'accord': 1568, 'freddi': 1569, 'brown': 1570, 'tim': 1571, 'seat': 1572, 'jeff': 1573, 'hitler': 1574, 'appropri': 1575, 'villag': 1576, 'unknown': 1577, 'knock': 1578, 'keaton': 1579, 'charg': 1580, 'unnecessari': 1581, 'media': 1582, 'england': 1583, 'enemi': 1584, 'empti': 1585, 'wave': 1586, 'utter': 1587, 'strength': 1588, 'perspect': 1589, 'dare': 1590, 'craft': 1591, 'buck': 1592, 'nativ': 1593, 'kiss': 1594, 'ford': 1595, 'correct': 1596, 'contrast': 1597, 'speed': 1598, 'soap': 1599, 'nazi': 1600, 'magnific': 1601, 'knowledg': 1602, 'distract': 1603, 'chill': 1604, 'anywher': 1605, 'mission': 1606, 'ice': 1607, 'fred': 1608, 'breath': 1609, '1980': 1610, 'moon': 1611, 'jr': 1612, 'joan': 1613, 'crowd': 1614, 'soft': 1615, 'kate': 1616, 'frighten': 1617, '000': 1618, 'nick': 1619, 'hundr': 1620, 'dick': 1621, 'dan': 1622, 'somebodi': 1623, 'simon': 1624, 'radio': 1625, 'dozen': 1626, 'thousand': 1627, 'shakespear': 1628, 'loss': 1629, 'andrew': 1630, 'academi': 1631, 'vehicl': 1632, 'sum': 1633, 'root': 1634, 'quot': 1635, 'account': 1636, 'leg': 1637, 'convent': 1638, 'behavior': 1639, '1970': 1640, 'regular': 1641, 'gold': 1642, 'worker': 1643, 'pretenti': 1644, 'demand': 1645, 'compet': 1646, 'stretch': 1647, 'privat': 1648, 'notabl': 1649, 'lynch': 1650, 'japan': 1651, 'interpret': 1652, 'explos': 1653, 'candi': 1654, 'tarzan': 1655, 'debut': 1656, 'constant': 1657, 'translat': 1658, 'spi': 1659, 'sea': 1660, 'revolv': 1661, 'prais': 1662, 'threaten': 1663, 'technolog': 1664, 'sat': 1665, 'quiet': 1666, 'jesu': 1667, 'franc': 1668, 'failur': 1669, 'ass': 1670, 'toy': 1671, 'punch': 1672, 'met': 1673, 'kevin': 1674, 'higher': 1675, 'aid': 1676, 'vh': 1677, 'mike': 1678, 'interact': 1679, 'abandon': 1680, 'separ': 1681, 'confront': 1682, 'command': 1683, 'bet': 1684, 'techniqu': 1685, 'stunt': 1686, 'site': 1687, 'servic': 1688, 'recal': 1689, 'gotten': 1690, 'belong': 1691, 'freak': 1692, 'foot': 1693, 'cabl': 1694, 'bug': 1695, 'jimmi': 1696, 'fu': 1697, 'capabl': 1698, 'bright': 1699, 'african': 1700, 'succeed': 1701, 'stock': 1702, 'presid': 1703, 'fat': 1704, 'clark': 1705, 'boat': 1706, 'structur': 1707, 'spanish': 1708, 'gene': 1709, 'paper': 1710, 'kidnap': 1711, 'whilst': 1712, 'factor': 1713, 'belief': 1714, 'witti': 1715, 'tree': 1716, 'realism': 1717, 'realis': 1718, 'educ': 1719, 'complic': 1720, 'bob': 1721, 'attend': 1722, 'santa': 1723, 'finest': 1724, 'broken': 1725, 'assist': 1726, 'v': 1727, 'up': 1728, 'smoke': 1729, 'observ': 1730, 'determin': 1731, 'depart': 1732, 'rubbish': 1733, 'routin': 1734, 'oper': 1735, 'lewi': 1736, 'hat': 1737, 'fame': 1738, 'domin': 1739, 'safe': 1740, 'morgan': 1741, 'lone': 1742, 'kinda': 1743, 'hook': 1744, 'foreign': 1745, 'advanc': 1746, 'rank': 1747, 'numer': 1748, 'werewolf': 1749, 'washington': 1750, 'vs': 1751, 'shape': 1752, 'shallow': 1753, 'rose': 1754, 'civil': 1755, 'morn': 1756, 'gari': 1757, 'winner': 1758, 'ordinari': 1759, 'kong': 1760, 'accomplish': 1761, 'whenev': 1762, 'virtual': 1763, 'peac': 1764, 'grab': 1765, 'offens': 1766, 'luck': 1767, 'h': 1768, 'welcom': 1769, 'unfunni': 1770, 'patient': 1771, 'contriv': 1772, 'complain': 1773, 'bigger': 1774, 'activ': 1775, 'trek': 1776, 'pretend': 1777, 'dimension': 1778, 'con': 1779, 'wake': 1780, 'lesbian': 1781, 'flash': 1782, 'eric': 1783, 'dri': 1784, 'code': 1785, 'cain': 1786, 'statu': 1787, 'manipul': 1788, 'guard': 1789, 'dancer': 1790, 'corrupt': 1791, 'albert': 1792, 'speech': 1793, 'sourc': 1794, 'signific': 1795, 'gain': 1796, 'context': 1797, 'awkward': 1798, 'sean': 1799, 'psycho': 1800, 'corni': 1801, 'clip': 1802, 'anthoni': 1803, '13': 1804, 'w': 1805, 'theatric': 1806, 'religion': 1807, 'reli': 1808, 'priest': 1809, 'curiou': 1810, 'advic': 1811, 'flow': 1812, 'addict': 1813, 'specif': 1814, 'skin': 1815, 'secur': 1816, 'jennif': 1817, 'howard': 1818, 'asian': 1819, 'promot': 1820, 'organ': 1821, 'luke': 1822, 'golden': 1823, 'core': 1824, 'comfort': 1825, 'lucki': 1826, 'cheat': 1827, 'cash': 1828, 'lower': 1829, 'dislik': 1830, 'associ': 1831, 'wing': 1832, 'spell': 1833, 'regret': 1834, 'frequent': 1835, 'frankli': 1836, 'devic': 1837, 'degre': 1838, 'contribut': 1839, 'balanc': 1840, 'sake': 1841, 'print': 1842, 'lake': 1843, 'forgiv': 1844, 'thoma': 1845, 'mass': 1846, 'betti': 1847, 'unexpect': 1848, 'gordon': 1849, 'crack': 1850, 'unfold': 1851, 'invit': 1852, 'grown': 1853, 'depend': 1854, 'construct': 1855, 'categori': 1856, 'amateur': 1857, 'walter': 1858, 'matur': 1859, 'intellectu': 1860, 'honor': 1861, 'grew': 1862, 'condit': 1863, 'anna': 1864, 'veteran': 1865, 'sudden': 1866, 'spectacular': 1867, 'sole': 1868, 'mirror': 1869, 'robin': 1870, 'overli': 1871, 'meanwhil': 1872, 'liner': 1873, 'grip': 1874, 'gift': 1875, 'freedom': 1876, 'experienc': 1877, 'demonstr': 1878, 'card': 1879, 'unabl': 1880, 'theori': 1881, 'subtitl': 1882, 'sheriff': 1883, 'section': 1884, 'oliv': 1885, 'drew': 1886, 'crappi': 1887, 'colour': 1888, 'circumst': 1889, 'brilliantli': 1890, 'sheer': 1891, 'pile': 1892, 'path': 1893, 'parker': 1894, 'matt': 1895, 'laughter': 1896, 'cook': 1897, 'altern': 1898, 'wander': 1899, 'treatment': 1900, 'sinatra': 1901, 'relief': 1902, 'lawyer': 1903, 'hall': 1904, 'defin': 1905, 'accident': 1906, 'hank': 1907, 'dragon': 1908, 'captiv': 1909, 'moor': 1910, 'halloween': 1911, 'gratuit': 1912, 'wound': 1913, 'wayn': 1914, 'unintent': 1915, 'kung': 1916, 'k': 1917, 'jacki': 1918, 'cowboy': 1919, 'broadway': 1920, 'barbara': 1921, 'winter': 1922, 'surreal': 1923, 'statement': 1924, 'spoof': 1925, 'canadian': 1926, 'treasur': 1927, 'gonna': 1928, 'fish': 1929, 'fare': 1930, 'compos': 1931, 'cheer': 1932, 'woodi': 1933, 'victor': 1934, 'unrealist': 1935, 'sensit': 1936, 'emerg': 1937, 'sympathet': 1938, 'ran': 1939, 'neighbor': 1940, 'driven': 1941, 'topic': 1942, 'overlook': 1943, 'menac': 1944, 'glass': 1945, 'expos': 1946, 'authent': 1947, 'michel': 1948, 'handsom': 1949, 'gross': 1950, 'chief': 1951, 'ancient': 1952, 'stranger': 1953, 'russel': 1954, 'pleasant': 1955, 'nevertheless': 1956, 'network': 1957, 'feet': 1958, 'contemporari': 1959, 'comedian': 1960, 'cinderella': 1961, 'built': 1962, 'underr': 1963, 'miser': 1964, 'letter': 1965, 'gori': 1966, 'endless': 1967, 'earn': 1968, 'consider': 1969, 'blockbust': 1970, 'switch': 1971, 'solv': 1972, 'brook': 1973, 'virgin': 1974, 'victoria': 1975, 'joseph': 1976, 'edward': 1977, 'convict': 1978, 'bullet': 1979, 'scenario': 1980, 'scale': 1981, 'cynic': 1982, 'chosen': 1983, 'alex': 1984, '0': 1985, 'sword': 1986, 'outrag': 1987, 'gut': 1988, 'curs': 1989, 'com': 1990, 'wrap': 1991, 'uk': 1992, 'substanc': 1993, 'screenwrit': 1994, 'proper': 1995, 'monkey': 1996, 'juli': 1997, 'driver': 1998, 'remov': 1999, 'par': 2000, 'indic': 2001, 'court': 2002, 'bird': 2003, 'roy': 2004, 'rental': 2005, 'nanci': 2006, 'naiv': 2007, 'loser': 2008, 'inevit': 2009, 'grave': 2010, 'consequ': 2011, 'advertis': 2012, 'slap': 2013, 'le': 2014, 'invis': 2015, 'germani': 2016, 'fatal': 2017, 'bridg': 2018, 'brave': 2019, 'provok': 2020, 'loui': 2021, 'footbal': 2022, 'anger': 2023, 'ador': 2024, 'chan': 2025, 'anderson': 2026, 'alcohol': 2027, 'willi': 2028, 'stumbl': 2029, 'ryan': 2030, 'professor': 2031, 'sharp': 2032, 'patrick': 2033, 'bat': 2034, 'australian': 2035, 'assassin': 2036, '1930': 2037, 'trilog': 2038, 'strongli': 2039, 'saturday': 2040, 'refresh': 2041, 'lousi': 2042, 'liber': 2043, 'heck': 2044, 'eight': 2045, 'deni': 2046, 'cell': 2047, 'ape': 2048, 'amateurish': 2049, 'sin': 2050, 'vagu': 2051, 'san': 2052, 'resid': 2053, 'justifi': 2054, 'terrifi': 2055, 'sympathi': 2056, 'reput': 2057, 'mini': 2058, 'indi': 2059, 'defeat': 2060, 'creator': 2061, 'tediou': 2062, 'task': 2063, 'tabl': 2064, 'prevent': 2065, 'expert': 2066, 'endur': 2067, 'trial': 2068, 'rival': 2069, 'offend': 2070, 'imit': 2071, 'employ': 2072, 'che': 2073, 'basebal': 2074, 'weekend': 2075, 'pitch': 2076, 'max': 2077, 'fairi': 2078, 'europ': 2079, 'dig': 2080, 'complaint': 2081, 'beach': 2082, 'risk': 2083, 'purchas': 2084, 'murphi': 2085, 'format': 2086, 'titan': 2087, 'tini': 2088, 'reminisc': 2089, 'powel': 2090, 'nois': 2091, 'hype': 2092, 'harsh': 2093, 'glimps': 2094, 'bite': 2095, 'till': 2096, 'strip': 2097, 'prime': 2098, 'north': 2099, 'fals': 2100, 'asleep': 2101, '14': 2102, 'texa': 2103, 'revel': 2104, 'destruct': 2105, 'descript': 2106, 'africa': 2107, 'uninterest': 2108, 'surfac': 2109, 'spin': 2110, 'sitcom': 2111, 'semi': 2112, 'inner': 2113, 'excess': 2114, 'arrest': 2115, 'twin': 2116, 'massiv': 2117, 'makeup': 2118, 'maintain': 2119, 'hitchcock': 2120, 'dinosaur': 2121, 'controversi': 2122, 'argu': 2123, 'stare': 2124, 'reject': 2125, 'melodrama': 2126, 'ludicr': 2127, 'kim': 2128, 'insist': 2129, 'ideal': 2130, 'expens': 2131, 'supernatur': 2132, 'subplot': 2133, 'press': 2134, 'nail': 2135, 'host': 2136, 'ga': 2137, 'forest': 2138, 'erot': 2139, 'columbo': 2140, 'atroci': 2141, 'ala': 2142, 'presum': 2143, 'notch': 2144, 'identifi': 2145, 'dude': 2146, 'cant': 2147, 'plagu': 2148, 'method': 2149, 'guest': 2150, 'forgett': 2151, 'crude': 2152, 'closer': 2153, 'character': 2154, 'princess': 2155, 'lion': 2156, 'landscap': 2157, 'foster': 2158, 'ear': 2159, 'border': 2160, 'beast': 2161, 'urban': 2162, 'storytel': 2163, 'previous': 2164, 'pacino': 2165, 'jungl': 2166, 'damag': 2167, 'bound': 2168, 'birth': 2169, 'aunt': 2170, 'accus': 2171, 'thirti': 2172, 'propaganda': 2173, 'nude': 2174, 'jess': 2175, 'guid': 2176, 'emma': 2177, 'doll': 2178, 'chose': 2179, 'whoever': 2180, 'warrior': 2181, 'pet': 2182, 'mate': 2183, 'mainstream': 2184, '25': 2185, 'upset': 2186, 'size': 2187, 'poster': 2188, 'merit': 2189, 'latest': 2190, 'gritti': 2191, 'friday': 2192, 'exact': 2193, 'deadli': 2194, 'cooper': 2195, 'wilson': 2196, 'warner': 2197, 'ton': 2198, 'sun': 2199, 'settl': 2200, 'rough': 2201, 'popul': 2202, 'corps': 2203, 'contest': 2204, 'contact': 2205, 'citizen': 2206, 'buff': 2207, 'blend': 2208, '1990': 2209, 'widow': 2210, 'select': 2211, 'rat': 2212, 'pitt': 2213, 'overcom': 2214, 'mgm': 2215, 'metal': 2216, 'environ': 2217, 'bu': 2218, 'alic': 2219, 'ted': 2220, 'revolut': 2221, 'particip': 2222, 'link': 2223, 'lift': 2224, 'guilti': 2225, 'prostitut': 2226, 'moron': 2227, 'matrix': 2228, 'johnson': 2229, 'exagger': 2230, 'corpor': 2231, 'corner': 2232, 'afternoon': 2233, 'accompani': 2234, '1960': 2235, 'sincer': 2236, 'multipl': 2237, 'leagu': 2238, 'instal': 2239, 'hood': 2240, 'holm': 2241, 'friendli': 2242, 'doom': 2243, 'clair': 2244, 'sunday': 2245, 'string': 2246, 'lugosi': 2247, 'junk': 2248, 'irish': 2249, 'hip': 2250, 'grim': 2251, 'examin': 2252, 'defend': 2253, 'campi': 2254, 'blah': 2255, 'aka': 2256, 'advis': 2257, 'varieti': 2258, 'tight': 2259, 'shut': 2260, 'shake': 2261, 'rachel': 2262, 'pro': 2263, 'icon': 2264, 'confid': 2265, 'sullivan': 2266, 'mexican': 2267, 'medic': 2268, 'jaw': 2269, 'goal': 2270, 'directli': 2271, 'denni': 2272, 'attach': 2273, 'vietnam': 2274, 'truck': 2275, 'terrorist': 2276, 'sentenc': 2277, 'sarah': 2278, 'prior': 2279, 'legendari': 2280, 'duke': 2281, 'dean': 2282, 'courag': 2283, 'breast': 2284, 'bourn': 2285, 'yell': 2286, 'un': 2287, 'split': 2288, 'proceed': 2289, 'nose': 2290, 'hong': 2291, 'entri': 2292, 'donald': 2293, 'behav': 2294, 'unconvinc': 2295, 'swim': 2296, 'stolen': 2297, 'lifetim': 2298, 'jerk': 2299, 'gather': 2300, 'forth': 2301, 'everywher': 2302, 'crush': 2303, 'confess': 2304, 'concentr': 2305, 'buri': 2306, 'borrow': 2307, 'turkey': 2308, 'spite': 2309, 'pan': 2310, 'lip': 2311, 'julia': 2312, 'deliveri': 2313, 'california': 2314, 'reward': 2315, 'quest': 2316, 'proud': 2317, 'offici': 2318, 'hoffman': 2319, 'freeman': 2320, 'flight': 2321, 'downright': 2322, 'china': 2323, 'worthwhil': 2324, 'sir': 2325, 'sink': 2326, 'notori': 2327, 'lazi': 2328, 'jon': 2329, 'jail': 2330, 'inept': 2331, 'fade': 2332, 'fabul': 2333, 'encourag': 2334, 'betray': 2335, 'teeth': 2336, 'susan': 2337, 'survivor': 2338, 'storm': 2339, 'shower': 2340, 'retard': 2341, 'relev': 2342, 'lisa': 2343, 'imageri': 2344, 'cousin': 2345, 'branagh': 2346, 'bell': 2347, 'bag': 2348, 'tremend': 2349, 'trade': 2350, 'toler': 2351, 'summari': 2352, 'stab': 2353, 'shark': 2354, 'quirki': 2355, 'mexico': 2356, 'hugh': 2357, 'finger': 2358, 'facial': 2359, 'bride': 2360, 'alright': 2361, 'von': 2362, 'pose': 2363, 'hyster': 2364, 'ha': 2365, 'blown': 2366, 'bitter': 2367, 'scheme': 2368, 'ron': 2369, 'ned': 2370, 'larri': 2371, 'cruel': 2372, 'christ': 2373, 'bone': 2374, 'afterward': 2375, 'address': 2376, 'traci': 2377, 'tour': 2378, 'thumb': 2379, 'swear': 2380, 'snake': 2381, 'screw': 2382, 'pursu': 2383, 'feed': 2384, 'distinct': 2385, 'beg': 2386, 'stomach': 2387, 'raw': 2388, 'photo': 2389, 'occas': 2390, 'obscur': 2391, 'mechan': 2392, 'chair': 2393, 'southern': 2394, 'sidney': 2395, 'resist': 2396, 'render': 2397, 'necessarili': 2398, 'holiday': 2399, 'heavili': 2400, 'hardi': 2401, 'gruesom': 2402, 'chain': 2403, 'cabin': 2404, 'argument': 2405, 'understood': 2406, 'satan': 2407, 'racist': 2408, 'philip': 2409, 'indulg': 2410, 'india': 2411, 'tongu': 2412, 'stalk': 2413, 'pregnant': 2414, 'outfit': 2415, 'obnoxi': 2416, 'midnight': 2417, 'lay': 2418, 'integr': 2419, 'fourth': 2420, 'forgot': 2421, 'belov': 2422, 'ticket': 2423, 'slapstick': 2424, 'restor': 2425, 'magazin': 2426, 'inhabit': 2427, 'garden': 2428, 'deeper': 2429, 'carol': 2430, '17': 2431, 'shoe': 2432, 'lincoln': 2433, 'incid': 2434, 'devot': 2435, 'brad': 2436, 'underground': 2437, 'sandler': 2438, 'maria': 2439, 'lili': 2440, 'guarante': 2441, 'elizabeth': 2442, 'divorc': 2443, 'disbelief': 2444, 'benefit': 2445, 'anticip': 2446, 'slave': 2447, 'princip': 2448, 'mildli': 2449, 'greater': 2450, 'explod': 2451, 'cring': 2452, 'creation': 2453, 'capit': 2454, 'bbc': 2455, 'amazingli': 2456, 'lesli': 2457, 'introduct': 2458, 'halfway': 2459, 'funnier': 2460, 'extraordinari': 2461, 'wreck': 2462, 'transfer': 2463, 'text': 2464, 'tap': 2465, 'punish': 2466, 'overwhelm': 2467, 'extent': 2468, 'enhanc': 2469, 'advantag': 2470, 'preview': 2471, 'plant': 2472, 'lo': 2473, 'lane': 2474, 'jessica': 2475, 'horrif': 2476, 'error': 2477, 'east': 2478, 'dynam': 2479, 'deliber': 2480, 'vincent': 2481, 'vacat': 2482, 'sophist': 2483, 'miscast': 2484, 'miller': 2485, 'homosexu': 2486, 'ensu': 2487, 'basi': 2488, 'appli': 2489, '2000': 2490, 'via': 2491, 'uncomfort': 2492, 'steel': 2493, 'spoken': 2494, 'sleazi': 2495, 'reed': 2496, 'measur': 2497, 'mansion': 2498, 'extend': 2499, 'elev': 2500, 'bollywood': 2501, 'stanley': 2502, 'savag': 2503, 'overact': 2504, 'mous': 2505, 'melt': 2506, 'hippi': 2507, 'goofi': 2508, 'fix': 2509, 'dentist': 2510, 'daili': 2511, 'conceiv': 2512, 'cathol': 2513, 'breathtak': 2514, 'blair': 2515, 'beer': 2516, 'assign': 2517, 'alter': 2518, 'succe': 2519, 'subsequ': 2520, 'sacrific': 2521, 'properli': 2522, 'oppos': 2523, 'nowaday': 2524, 'inspector': 2525, 'everyday': 2526, 'carpent': 2527, 'burt': 2528, 'neck': 2529, 'massacr': 2530, 'laura': 2531, 'circl': 2532, 'block': 2533, 'seagal': 2534, 'portrait': 2535, 'pool': 2536, 'mob': 2537, 'lesser': 2538, 'grey': 2539, 'fay': 2540, 'fallen': 2541, 'concert': 2542, 'christi': 2543, 'access': 2544, 'usa': 2545, 'sinist': 2546, 'relax': 2547, 'react': 2548, 'jewish': 2549, 'jake': 2550, 'isol': 2551, 'competit': 2552, 'chees': 2553, 'suitabl': 2554, 'stink': 2555, 'spiritu': 2556, 'nonetheless': 2557, 'nine': 2558, 'lyric': 2559, 'ironi': 2560, 'immens': 2561, 'creep': 2562, 'chop': 2563, 'appal': 2564, '2006': 2565, 'user': 2566, 'spring': 2567, 'sold': 2568, 'showcas': 2569, 'shirt': 2570, 'retir': 2571, 'reduc': 2572, 'rage': 2573, 'nut': 2574, 'needless': 2575, 'navi': 2576, 'luci': 2577, 'franchis': 2578, 'adopt': 2579, 'zone': 2580, 'uninspir': 2581, 'stanwyck': 2582, 'per': 2583, 'nurs': 2584, 'jay': 2585, 'digit': 2586, 'bulli': 2587, 'bath': 2588, 'asham': 2589, 'upper': 2590, 'sutherland': 2591, 'oddli': 2592, 'laid': 2593, 'illustr': 2594, 'broadcast': 2595, 'amongst': 2596, '2001': 2597, '1940': 2598, 'throat': 2599, 'stylish': 2600, 'fulfil': 2601, 'disguis': 2602, 'brando': 2603, 'baker': 2604, 'aspir': 2605, 'wwii': 2606, 'wanna': 2607, 'thief': 2608, 'pride': 2609, 'pound': 2610, 'nobl': 2611, 'neighborhood': 2612, 'impli': 2613, 'endear': 2614, 'em': 2615, '18': 2616, 'tens': 2617, 'shoulder': 2618, 'shift': 2619, 'rochest': 2620, 'prop': 2621, 'distribut': 2622, 'diseas': 2623, 'dinner': 2624, 'dawn': 2625, 'coher': 2626, 'cinematograph': 2627, 'bo': 2628, 'bett': 2629, 'albeit': 2630, '16': 2631, 'wash': 2632, 'surf': 2633, 'snow': 2634, 'silenc': 2635, 'shout': 2636, 'rebel': 2637, 'poignant': 2638, 'matthau': 2639, 'knife': 2640, 'function': 2641, 'forti': 2642, 'contract': 2643, 'widmark': 2644, 'silver': 2645, 'reunion': 2646, 'proof': 2647, 'mindless': 2648, 'internet': 2649, 'instinct': 2650, 'horrend': 2651, 'henc': 2652, 'height': 2653, 'heat': 2654, 'elvira': 2655, 'eeri': 2656, 'duti': 2657, 'derek': 2658, 'chuck': 2659, 'cannib': 2660, 'cancel': 2661, 'torn': 2662, 'spielberg': 2663, 'repetit': 2664, 'premier': 2665, 'pie': 2666, 'neat': 2667, 'musician': 2668, 'mill': 2669, 'innov': 2670, 'incoher': 2671, 'greatli': 2672, 'glori': 2673, 'etern': 2674, 'elvi': 2675, 'alik': 2676, 'absorb': 2677, 'wealthi': 2678, 'trite': 2679, 'redempt': 2680, 'racism': 2681, 'precis': 2682, 'nelson': 2683, 'lovabl': 2684, 'itali': 2685, 'infam': 2686, 'horrifi': 2687, 'homag': 2688, 'fbi': 2689, 'diamond': 2690, 'crisi': 2691, 'burton': 2692, 'britain': 2693, 'blank': 2694, 'bang': 2695, 'announc': 2696, 'wilder': 2697, 'streisand': 2698, 'resolut': 2699, 'pat': 2700, 'parallel': 2701, 'helen': 2702, 'happili': 2703, 'hammer': 2704, 'flop': 2705, 'ensembl': 2706, 'dedic': 2707, 'chaplin': 2708, 'triumph': 2709, 'st': 2710, 'plastic': 2711, 'oil': 2712, 'mar': 2713, 'factori': 2714, 'disagre': 2715, 'cube': 2716, 'conclud': 2717, 'carter': 2718, 'broke': 2719, 'weight': 2720, 'vega': 2721, 'row': 2722, 'rocket': 2723, 'own': 2724, 'march': 2725, 'fighter': 2726, 'climb': 2727, 'chuckl': 2728, 'bush': 2729, 'wherea': 2730, 'unforgett': 2731, 'thug': 2732, 'spare': 2733, 'sensibl': 2734, 'mst3k': 2735, 'meaning': 2736, 'lust': 2737, 'luca': 2738, 'kurt': 2739, 'enorm': 2740, 'dump': 2741, 'dane': 2742, 'boot': 2743, 'threat': 2744, 'stress': 2745, 'rap': 2746, 'karloff': 2747, 'fifti': 2748, 'engin': 2749, 'difficulti': 2750, 'dear': 2751, 'caricatur': 2752, 'butt': 2753, 'brand': 2754, 'bobbi': 2755, 'arnold': 2756, 'adequ': 2757, 'swing': 2758, 'secretari': 2759, 'ralph': 2760, 'polish': 2761, 'journalist': 2762, 'homeless': 2763, 'hamlet': 2764, 'flynn': 2765, 'fest': 2766, 'elabor': 2767, 'ego': 2768, 'barri': 2769, 'arrog': 2770, 'unbear': 2771, 'tool': 2772, 'spike': 2773, 'simpson': 2774, 'resort': 2775, 'puppet': 2776, 'induc': 2777, 'grate': 2778, 'float': 2779, 'fanci': 2780, 'conspiraci': 2781, 'arrang': 2782, 'tribut': 2783, 'pig': 2784, 'phillip': 2785, 'muppet': 2786, 'guilt': 2787, 'exercis': 2788, 'cruis': 2789, 'choreograph': 2790, 'boll': 2791, 'basement': 2792, 'ward': 2793, 'tower': 2794, 'toilet': 2795, 'stan': 2796, 'slip': 2797, 'scarecrow': 2798, 'puzzl': 2799, 'medium': 2800, 'layer': 2801, 'korean': 2802, 'item': 2803, 'ham': 2804, 'file': 2805, 'fianc': 2806, 'editor': 2807, 'document': 2808, 'babe': 2809, '24': 2810, 'transit': 2811, 'territori': 2812, 'superfici': 2813, 'spark': 2814, 'slaughter': 2815, 'portion': 2816, 'philosoph': 2817, 'persona': 2818, 'orient': 2819, 'minim': 2820, 'librari': 2821, 'larger': 2822, 'inexplic': 2823, 'glover': 2824, 'doc': 2825, 'denzel': 2826, 'catherin': 2827, 'assur': 2828, 'wolf': 2829, 'walken': 2830, 'sneak': 2831, 'shi': 2832, 'pg': 2833, 'owe': 2834, 'jet': 2835, 'jeremi': 2836, 'financi': 2837, 'dorothi': 2838, 'curti': 2839, 'boredom': 2840, 'ban': 2841, 'whale': 2842, 'profound': 2843, 'multi': 2844, 'metaphor': 2845, 'hudson': 2846, 'eleph': 2847, 'cusack': 2848, 'backdrop': 2849, 'ambigu': 2850, 'viru': 2851, 'union': 2852, 'ultra': 2853, 'stiff': 2854, 'rave': 2855, 'notion': 2856, 'implaus': 2857, 'hack': 2858, 'gadget': 2859, 'elsewher': 2860, 'birthday': 2861, '2005': 2862, 'urg': 2863, 'superhero': 2864, 'squar': 2865, 'slight': 2866, 'reader': 2867, 'poison': 2868, 'pad': 2869, 'newspap': 2870, 'lloyd': 2871, 'hawk': 2872, 'eva': 2873, 'eastwood': 2874, 'distanc': 2875, 'disc': 2876, 'deriv': 2877, 'canada': 2878, 'bibl': 2879, 'afford': 2880, '1st': 2881, 'spread': 2882, 'skit': 2883, 'sadist': 2884, 'restaur': 2885, 'montag': 2886, 'huh': 2887, 'heston': 2888, 'health': 2889, 'essenc': 2890, 'drown': 2891, 'cure': 2892, 'charisma': 2893, 'button': 2894, 'scoobi': 2895, 'peak': 2896, 'muslim': 2897, 'maniac': 2898, 'lab': 2899, 'invest': 2900, 'gradual': 2901, 'godfath': 2902, 'fetch': 2903, 'estat': 2904, 'dealt': 2905, 'companion': 2906, 'tea': 2907, 'subtleti': 2908, 'servant': 2909, 'ritter': 2910, 'miik': 2911, 'kane': 2912, 'gothic': 2913, 'cup': 2914, 'countless': 2915, 'alli': 2916, 'salli': 2917, 'iii': 2918, 'heroic': 2919, 'electr': 2920, 'elect': 2921, 'charismat': 2922, 'briefli': 2923, 'wannab': 2924, 'toss': 2925, 'tender': 2926, 'resourc': 2927, 'reel': 2928, 'nuanc': 2929, 'neil': 2930, 'ingredi': 2931, 'grandmoth': 2932, 'cole': 2933, 'bud': 2934, 'admittedli': 2935, 'stronger': 2936, 'stood': 2937, 'shall': 2938, 'reev': 2939, 'punk': 2940, 'poverti': 2941, 'pit': 2942, 'pauli': 2943, 'mild': 2944, 'mafia': 2945, 'label': 2946, 'kubrick': 2947, 'gate': 2948, 'dawson': 2949, 'carrey': 2950, 'useless': 2951, 'updat': 2952, 'terri': 2953, 'tag': 2954, 'smooth': 2955, 'smash': 2956, 'outcom': 2957, 'ian': 2958, 'fond': 2959, 'easier': 2960, 'cox': 2961, 'cardboard': 2962, 'burst': 2963, 'bakshi': 2964, 'astair': 2965, 'assault': 2966, 'vulner': 2967, 'vari': 2968, 'sketch': 2969, 'samurai': 2970, 'rex': 2971, 'resolv': 2972, 'qualifi': 2973, 'melodramat': 2974, 'increasingli': 2975, 'fist': 2976, 'exchang': 2977, 'divers': 2978, 'coincid': 2979, '2002': 2980, 'templ': 2981, 'tame': 2982, 'suspend': 2983, 'scratch': 2984, 'reynold': 2985, 'luckili': 2986, 'insert': 2987, 'conveni': 2988, 'brillianc': 2989, 'blast': 2990, 'be': 2991, 'walker': 2992, 'strictli': 2993, 'soprano': 2994, 'seventi': 2995, 'pin': 2996, 'nuclear': 2997, 'meat': 2998, 'matthew': 2999, 'jami': 3000, 'hamilton': 3001, 'gotta': 3002, 'fisher': 3003, 'farm': 3004, 'coach': 3005, 'ambiti': 3006, 'worthless': 3007, 'timeless': 3008, 'struck': 3009, 'spooki': 3010, 'revers': 3011, 'recreat': 3012, 'ninja': 3013, 'monk': 3014, 'kudo': 3015, 'joey': 3016, 'instantli': 3017, 'grasp': 3018, 'empir': 3019, 'eccentr': 3020, 'discoveri': 3021, 'convolut': 3022, 'closet': 3023, 'clock': 3024, 'cave': 3025, 'butcher': 3026, 'brosnan': 3027, 'wipe': 3028, 'sloppi': 3029, 'sidekick': 3030, 'seller': 3031, 'selfish': 3032, 'partli': 3033, 'pal': 3034, 'norman': 3035, 'mitchel': 3036, 'miracl': 3037, 'inconsist': 3038, 'importantli': 3039, 'gray': 3040, 'fifteen': 3041, 'evok': 3042, 'eighti': 3043, 'declar': 3044, 'communist': 3045, 'clown': 3046, 'cliff': 3047, 'bleak': 3048, 'websit': 3049, 'superbl': 3050, 'stoog': 3051, 'seed': 3052, 'psychiatrist': 3053, 'piano': 3054, 'lifestyl': 3055, 'ho': 3056, 'flawless': 3057, 'farc': 3058, 'enthusiast': 3059, 'destin': 3060, 'debat': 3061, 'chew': 3062, 'cheek': 3063, 'australia': 3064, 'aforement': 3065, '45': 3066, 'wrestl': 3067, 'wick': 3068, 'splatter': 3069, 'soviet': 3070, 'slice': 3071, 'regardless': 3072, 'pressur': 3073, 'kitchen': 3074, 'incompet': 3075, 'emili': 3076, 'drivel': 3077, 'directori': 3078, 'dire': 3079, 'dash': 3080, 'bash': 3081, 'anni': 3082, 'akshay': 3083, 'abc': 3084, 'suppli': 3085, 'seduc': 3086, 'recov': 3087, 'prize': 3088, 'pleasantli': 3089, 'mann': 3090, 'lou': 3091, 'ken': 3092, 'judi': 3093, 'jar': 3094, 'increas': 3095, 'helicopt': 3096, 'glow': 3097, 'flower': 3098, 'duo': 3099, 'doo': 3100, 'distant': 3101, 'dave': 3102, 'curios': 3103, 'cia': 3104, 'chapter': 3105, 'cameron': 3106, 'cagney': 3107, 'boil': 3108, 'blob': 3109, 'beaten': 3110, 'artifici': 3111, 'web': 3112, 'turner': 3113, 'splendid': 3114, 'ranger': 3115, 'psychot': 3116, 'perri': 3117, 'panic': 3118, 'laurel': 3119, 'hop': 3120, 'goldberg': 3121, 'glenn': 3122, 'francisco': 3123, 'favour': 3124, 'ellen': 3125, 'eleg': 3126, 'drunken': 3127, 'craven': 3128, 'craig': 3129, 'combat': 3130, 'wizard': 3131, 'slightest': 3132, 'shortli': 3133, 'ruth': 3134, 'rid': 3135, 'plausibl': 3136, 'philosophi': 3137, 'modesti': 3138, 'min': 3139, 'hatr': 3140, 'greek': 3141, 'graduat': 3142, 'gentl': 3143, 'gandhi': 3144, 'fx': 3145, 'flip': 3146, 'falk': 3147, 'alexand': 3148, '20th': 3149, 'we': 3150, 'unpleas': 3151, 'tall': 3152, 'preciou': 3153, 'ocean': 3154, 'manhattan': 3155, 'lend': 3156, 'legal': 3157, 'knight': 3158, 'jealou': 3159, 'holi': 3160, 'harm': 3161, 'futurist': 3162, 'fund': 3163, 'felix': 3164, 'dracula': 3165, 'thread': 3166, 'tank': 3167, 'scientif': 3168, 'reviv': 3169, 'overdon': 3170, 'nod': 3171, 'mock': 3172, 'giallo': 3173, 'forbidden': 3174, 'explicit': 3175, 'digniti': 3176, 'childish': 3177, 'bless': 3178, 'ami': 3179, 'yesterday': 3180, 'verhoeven': 3181, 'unwatch': 3182, 'torment': 3183, 'thick': 3184, 'repeatedli': 3185, 'pirat': 3186, 'nerv': 3187, 'mel': 3188, 'margaret': 3189, 'fever': 3190, 'eve': 3191, 'elderli': 3192, 'broad': 3193, 'awe': 3194, 'awaken': 3195, '99': 3196, '2004': 3197, 'uniform': 3198, 'timothi': 3199, 'stiller': 3200, 'royal': 3201, 'romero': 3202, 'roman': 3203, 'rivet': 3204, 'publish': 3205, 'politician': 3206, 'lean': 3207, 'launch': 3208, 'kay': 3209, 'griffith': 3210, 'eas': 3211, 'custom': 3212, 'bin': 3213, 'automat': 3214, 'ambit': 3215, 'ah': 3216, 'acclaim': 3217, 'absenc': 3218, 'warren': 3219, 'wallac': 3220, 'transport': 3221, 'tomato': 3222, 'termin': 3223, 'sunshin': 3224, 'stinker': 3225, 'purpl': 3226, 'pulp': 3227, 'pierc': 3228, 'phrase': 3229, 'homicid': 3230, 'gabriel': 3231, 'foul': 3232, 'darker': 3233, 'crook': 3234, 'bathroom': 3235, 'antic': 3236, 'viciou': 3237, 'sixti': 3238, 'saint': 3239, 'revolutionari': 3240, 'rambo': 3241, 'q': 3242, 'prom': 3243, 'pray': 3244, 'packag': 3245, 'ought': 3246, 'marin': 3247, 'li': 3248, 'kenneth': 3249, 'karen': 3250, 'juvenil': 3251, 'horrid': 3252, 'hollow': 3253, 'eyr': 3254, 'evolv': 3255, 'donna': 3256, 'contrari': 3257, 'coloni': 3258, 'choreographi': 3259, 'brazil': 3260, 'awak': 3261, 'album': 3262, '2003': 3263, 'twelv': 3264, 'stole': 3265, 'ramon': 3266, 'overr': 3267, 'option': 3268, 'nerd': 3269, 'mummi': 3270, 'mildr': 3271, 'kapoor': 3272, 'ireland': 3273, 'dose': 3274, 'defi': 3275, 'conserv': 3276, 'candid': 3277, 'boast': 3278, 'blade': 3279, 'beatti': 3280, 'trio': 3281, 'protest': 3282, 'natali': 3283, 'kirk': 3284, 'jazz': 3285, 'global': 3286, 'funer': 3287, 'fulci': 3288, 'flame': 3289, 'detract': 3290, 'confirm': 3291, 'collabor': 3292, 'astonish': 3293, 'altman': 3294, 'yellow': 3295, 'whip': 3296, 'tommi': 3297, 'spit': 3298, 'shade': 3299, 'racial': 3300, 'nicholson': 3301, 'mystic': 3302, 'leap': 3303, 'enterpris': 3304, 'destini': 3305, 'delici': 3306, 'bull': 3307, 'bottl': 3308, 'blake': 3309, 'audio': 3310, 'vivid': 3311, 'visibl': 3312, 'todd': 3313, 'threw': 3314, 'swedish': 3315, 'staff': 3316, 'reunit': 3317, 'pseudo': 3318, 'popcorn': 3319, 'neo': 3320, 'merci': 3321, 'meaningless': 3322, 'inherit': 3323, 'harder': 3324, 'fonda': 3325, 'enchant': 3326, 'bedroom': 3327, 'altogeth': 3328, 'adolesc': 3329, 'wire': 3330, 'voight': 3331, 'uneven': 3332, 'tip': 3333, 'synopsi': 3334, 'suspici': 3335, 'ruthless': 3336, 'roommat': 3337, 'respond': 3338, 'reserv': 3339, 'moodi': 3340, 'madonna': 3341, 'leonard': 3342, 'lemmon': 3343, 'lawrenc': 3344, 'kennedi': 3345, 'jew': 3346, 'fanat': 3347, 'exhibit': 3348, 'edi': 3349, 'decor': 3350, 'crocodil': 3351, 'bust': 3352, 'befriend': 3353, 'await': 3354, 'atlanti': 3355, 'voyag': 3356, 'ventur': 3357, 'unsettl': 3358, 'rural': 3359, 'palma': 3360, 'incident': 3361, 'holli': 3362, 'garner': 3363, 'dimens': 3364, 'clumsi': 3365, 'clint': 3366, 'chao': 3367, 'centr': 3368, 'carl': 3369, 'bradi': 3370, 'bold': 3371, 'bargain': 3372, 'audit': 3373, 'abysm': 3374, '2007': 3375, 'wealth': 3376, 'versu': 3377, 'troop': 3378, 'trail': 3379, 'timon': 3380, 'tiger': 3381, 'poetic': 3382, 'neglect': 3383, 'nearbi': 3384, 'mall': 3385, 'lit': 3386, 'imperson': 3387, 'immigr': 3388, 'humili': 3389, 'hart': 3390, 'elimin': 3391, 'echo': 3392, 'daddi': 3393, 'cuba': 3394, 'characterist': 3395, 'cd': 3396, 'cari': 3397, 'ant': 3398, 'acknowledg': 3399, '2nd': 3400, 'solo': 3401, 'saga': 3402, 'repuls': 3403, 'pun': 3404, 'prejudic': 3405, 'paus': 3406, 'mistaken': 3407, 'mickey': 3408, 'marshal': 3409, 'jeffrey': 3410, 'infect': 3411, 'homer': 3412, 'domest': 3413, 'collaps': 3414, 'celluloid': 3415, 'undoubtedli': 3416, 'tribe': 3417, 'sore': 3418, 'promin': 3419, 'pant': 3420, 'olivi': 3421, 'milk': 3422, 'leon': 3423, 'interrupt': 3424, 'inappropri': 3425, 'inan': 3426, 'hbo': 3427, 'harvey': 3428, 'ginger': 3429, 'gear': 3430, 'equip': 3431, 'coffe': 3432, 'coat': 3433, 'chest': 3434, 'cake': 3435, 'assembl': 3436, 'apolog': 3437, '1996': 3438, 'vulgar': 3439, 'trace': 3440, 'solut': 3441, 'retain': 3442, 'primari': 3443, 'pot': 3444, 'polanski': 3445, 'pen': 3446, 'maggi': 3447, 'jenni': 3448, 'institut': 3449, 'instant': 3450, 'humbl': 3451, 'highest': 3452, 'furthermor': 3453, 'florida': 3454, 'exot': 3455, 'embrac': 3456, 'devast': 3457, 'consum': 3458, 'colonel': 3459, 'colleagu': 3460, 'brooklyn': 3461, 'aveng': 3462, 'airplan': 3463, 'ya': 3464, 'wive': 3465, 'strain': 3466, 'smaller': 3467, 'seduct': 3468, 'sale': 3469, 'rick': 3470, 'principl': 3471, 'poke': 3472, 'outer': 3473, 'linda': 3474, 'illog': 3475, 'godzilla': 3476, 'gender': 3477, 'dutch': 3478, 'disabl': 3479, 'dian': 3480, 'descend': 3481, 'cope': 3482, 'bowl': 3483, '3rd': 3484, '1999': 3485, 'yard': 3486, 'vast': 3487, 'secondli': 3488, 'scope': 3489, 'rabbit': 3490, 'primarili': 3491, 'predecessor': 3492, 'mixtur': 3493, 'lol': 3494, 'inferior': 3495, 'hal': 3496, 'gundam': 3497, 'gloriou': 3498, 'glamor': 3499, 'dud': 3500, 'dive': 3501, 'devoid': 3502, 'cue': 3503, 'bubbl': 3504, 'blatant': 3505, 'beneath': 3506, 'z': 3507, 'trademark': 3508, 'talki': 3509, 'streep': 3510, 'simplist': 3511, 'shirley': 3512, 'shelf': 3513, 'senseless': 3514, 'pearl': 3515, 'myer': 3516, 'museum': 3517, 'invas': 3518, 'hideou': 3519, 'grinch': 3520, 'garbo': 3521, 'et': 3522, 'domino': 3523, 'disjoint': 3524, 'countrysid': 3525, 'casual': 3526, 'breed': 3527, 'arab': 3528, 'april': 3529, 'alfr': 3530, 'alert': 3531, 'aggress': 3532, 'vanish': 3533, 'uwe': 3534, 'unhappi': 3535, 'stir': 3536, 'stellar': 3537, 'stack': 3538, 'slide': 3539, 'sh': 3540, 'robinson': 3541, 'robberi': 3542, 'rendit': 3543, 'oz': 3544, 'obtain': 3545, 'mayor': 3546, 'mail': 3547, 'maci': 3548, 'loyal': 3549, 'khan': 3550, 'illeg': 3551, 'hopeless': 3552, 'hardcor': 3553, 'experiment': 3554, 'disgrac': 3555, 'defens': 3556, 'boom': 3557, 'applaud': 3558, 'acid': 3559, 'wont': 3560, 'topless': 3561, 'tenant': 3562, 'tempt': 3563, 'spider': 3564, 'span': 3565, 'soccer': 3566, 'scroog': 3567, 'rifl': 3568, 'recruit': 3569, 'psychic': 3570, 'incomprehens': 3571, 'hartley': 3572, 'grandfath': 3573, 'fri': 3574, 'emphasi': 3575, 'dismiss': 3576, 'dicken': 3577, 'diana': 3578, 'declin': 3579, 'craze': 3580, 'counter': 3581, 'blew': 3582, 'berlin': 3583, 'amanda': 3584, 'woo': 3585, 'wet': 3586, 'trashi': 3587, 'sympath': 3588, 'sibl': 3589, 'shed': 3590, 'shaw': 3591, 'riot': 3592, 'revolt': 3593, 'resurrect': 3594, 'ration': 3595, 'porno': 3596, 'parad': 3597, 'niro': 3598, 'lumet': 3599, 'justin': 3600, 'intim': 3601, 'goer': 3602, 'faster': 3603, 'ethnic': 3604, 'bitch': 3605, 'worm': 3606, 'wheel': 3607, 'wendi': 3608, 'weakest': 3609, 'unreal': 3610, 'steam': 3611, 'slick': 3612, 'rider': 3613, 'region': 3614, 'patriot': 3615, 'partial': 3616, 'nephew': 3617, 'mario': 3618, 'lena': 3619, 'jonathan': 3620, 'immort': 3621, 'hopper': 3622, 'honesti': 3623, 'hesit': 3624, 'gap': 3625, 'feminist': 3626, 'farmer': 3627, 'ensur': 3628, 'enlighten': 3629, 'eager': 3630, 'dealer': 3631, 'commend': 3632, 'choru': 3633, 'biographi': 3634, 'ballet': 3635, 'andr': 3636, '00': 3637, 'wore': 3638, 'victori': 3639, 'vice': 3640, 'util': 3641, 'snap': 3642, 'skull': 3643, 'similarli': 3644, 'sappi': 3645, 'sandra': 3646, 'safeti': 3647, 'repress': 3648, 'psychopath': 3649, 'properti': 3650, 'prequel': 3651, 'owen': 3652, 'nostalg': 3653, 'mutant': 3654, 'morri': 3655, 'macarthur': 3656, 'leo': 3657, 'kingdom': 3658, 'hung': 3659, 'franco': 3660, 'confin': 3661, 'composit': 3662, 'charlott': 3663, 'blunt': 3664, 'whoopi': 3665, 'valuabl': 3666, 'thru': 3667, 'tail': 3668, 'tad': 3669, 'strand': 3670, 'speci': 3671, 'snl': 3672, 'rope': 3673, 'rocki': 3674, 'repli': 3675, 'recycl': 3676, 'rambl': 3677, 'pattern': 3678, 'nervou': 3679, 'montana': 3680, 'miseri': 3681, 'latin': 3682, 'kyle': 3683, 'hyde': 3684, 'heartbreak': 3685, 'farrel': 3686, 'exit': 3687, 'emperor': 3688, 'dust': 3689, 'drum': 3690, 'drain': 3691, 'despair': 3692, 'del': 3693, 'deed': 3694, 'dalton': 3695, 'compens': 3696, 'compass': 3697, 'cg': 3698, 'campbel': 3699, 'bumbl': 3700, 'bow': 3701, 'bonu': 3702, 'bergman': 3703, 'acquir': 3704, '1972': 3705, 'wacki': 3706, 'tonight': 3707, 'slug': 3708, 'rotten': 3709, 'roth': 3710, 'romp': 3711, 'rapist': 3712, 'radic': 3713, 'pour': 3714, 'percept': 3715, 'orson': 3716, 'oppress': 3717, 'olli': 3718, 'mistress': 3719, 'martian': 3720, 'gimmick': 3721, 'gal': 3722, 'downhil': 3723, 'da': 3724, 'contempl': 3725, 'chess': 3726, 'carradin': 3727, 'bleed': 3728, 'airport': 3729, '35': 3730, 'unpredict': 3731, 'tooth': 3732, 'taught': 3733, 'tackl': 3734, 'stilt': 3735, 'slash': 3736, 'shelley': 3737, 'pursuit': 3738, 'programm': 3739, 'preach': 3740, 'pervert': 3741, 'pervers': 3742, 'paltrow': 3743, 'mislead': 3744, 'melodi': 3745, 'heal': 3746, 'edgar': 3747, 'dazzl': 3748, 'champion': 3749, 'belt': 3750, 'banal': 3751, 'attorney': 3752, 'arguabl': 3753, 'arc': 3754, '1983': 3755, 'vocal': 3756, 'virginia': 3757, 'vengeanc': 3758, 'uplift': 3759, 'tiresom': 3760, 'sensat': 3761, 'rubi': 3762, 'raymond': 3763, 'poem': 3764, 'plight': 3765, 'passeng': 3766, 'orang': 3767, 'mesmer': 3768, 'marti': 3769, 'maid': 3770, 'graham': 3771, 'gambl': 3772, 'franki': 3773, 'employe': 3774, 'duval': 3775, 'dixon': 3776, 'conneri': 3777, 'closest': 3778, 'cleverli': 3779, 'chicken': 3780, 'bela': 3781, 'yawn': 3782, 'whine': 3783, 'volum': 3784, 'tube': 3785, 'swallow': 3786, 'suffic': 3787, 'sirk': 3788, 'secretli': 3789, 'scottish': 3790, 'quarter': 3791, 'profan': 3792, 'pokemon': 3793, 'paranoia': 3794, 'outing': 3795, 'numb': 3796, 'mute': 3797, 'monologu': 3798, 'lundgren': 3799, 'iran': 3800, 'inject': 3801, 'habit': 3802, 'giggl': 3803, 'gerard': 3804, 'extens': 3805, 'engross': 3806, 'crystal': 3807, 'convincingli': 3808, 'clone': 3809, 'climact': 3810, 'calm': 3811, 'bay': 3812, 'amitabh': 3813, 'abraham': 3814, '1968': 3815, 'underst': 3816, 'trend': 3817, 'taxi': 3818, 'surpass': 3819, 'spock': 3820, 'septemb': 3821, 'richardson': 3822, 'profess': 3823, 'poetri': 3824, 'plod': 3825, 'nichola': 3826, 'meander': 3827, 'lowest': 3828, 'linger': 3829, 'junior': 3830, 'im': 3831, 'grotesqu': 3832, 'frankenstein': 3833, 'franci': 3834, 'fed': 3835, 'expand': 3836, 'ethan': 3837, 'earl': 3838, 'dispos': 3839, 'chicago': 3840, 'bend': 3841, 'backward': 3842, 'austen': 3843, 'abort': 3844, 'waitress': 3845, 'tourist': 3846, 'sue': 3847, 'stallon': 3848, 'spoke': 3849, 'simplic': 3850, 'rubber': 3851, 'rant': 3852, 'nostalgia': 3853, 'myth': 3854, 'mundan': 3855, 'muddl': 3856, 'lure': 3857, 'literatur': 3858, 'instrument': 3859, 'hum': 3860, 'household': 3861, 'greedi': 3862, 'eugen': 3863, 'econom': 3864, 'dysfunct': 3865, 'descent': 3866, 'der': 3867, 'compliment': 3868, 'catchi': 3869, 'cannon': 3870, 'stale': 3871, 'sissi': 3872, 'recognit': 3873, 'recognis': 3874, 'randi': 3875, 'phoni': 3876, 'phantom': 3877, 'omen': 3878, 'occupi': 3879, 'mortal': 3880, 'molli': 3881, 'map': 3882, 'mankind': 3883, 'louis': 3884, 'lang': 3885, 'june': 3886, 'irrelev': 3887, 'insur': 3888, 'hello': 3889, 'furi': 3890, 'flee': 3891, 'firstli': 3892, 'equival': 3893, 'eaten': 3894, 'duck': 3895, 'dictat': 3896, 'dement': 3897, 'deaf': 3898, 'damon': 3899, 'crucial': 3900, 'coast': 3901, 'cent': 3902, 'carel': 3903, 'bacal': 3904, 'alongsid': 3905, 'wisdom': 3906, 'twilight': 3907, 'rude': 3908, 'rooney': 3909, 'reign': 3910, 'onlin': 3911, 'newli': 3912, 'loyalti': 3913, 'likewis': 3914, 'lengthi': 3915, 'labor': 3916, 'heel': 3917, 'grayson': 3918, 'freez': 3919, 'dreari': 3920, 'drake': 3921, 'distinguish': 3922, 'damm': 3923, 'daisi': 3924, 'cyborg': 3925, 'bump': 3926, 'buffalo': 3927, 'blackmail': 3928, 'biko': 3929, 'bike': 3930, 'ashley': 3931, 'antwon': 3932, '1973': 3933, 'worn': 3934, 'vein': 3935, 'unorigin': 3936, 'tunnel': 3937, 'startl': 3938, 'sailor': 3939, 'ridden': 3940, 'provoc': 3941, 'proce': 3942, 'prey': 3943, 'pink': 3944, 'nineti': 3945, 'keith': 3946, 'interior': 3947, 'inher': 3948, 'incorpor': 3949, 'exposur': 3950, 'emphas': 3951, 'chronicl': 3952, 'butler': 3953, 'boxer': 3954, 'basketbal': 3955, 'barrymor': 3956, 'baddi': 3957, 'attribut': 3958, 'approv': 3959, 'analysi': 3960, 'walsh': 3961, 'unrel': 3962, 'underli': 3963, 'undeni': 3964, 'substitut': 3965, 'stalker': 3966, 'simmon': 3967, 'robbin': 3968, 'predat': 3969, 'othello': 3970, 'nicol': 3971, 'mormon': 3972, 'millionair': 3973, 'mighti': 3974, 'meyer': 3975, 'meg': 3976, 'julian': 3977, 'indiffer': 3978, 'improvis': 3979, 'hypnot': 3980, 'fleet': 3981, 'er': 3982, 'elm': 3983, 'drift': 3984, 'degrad': 3985, 'condemn': 3986, 'carla': 3987, 'bunni': 3988, 'belushi': 3989, 'barrel': 3990, 'watson': 3991, 'warmth': 3992, 'vital': 3993, 'unawar': 3994, 'shove': 3995, 'rukh': 3996, 'roof': 3997, 'reid': 3998, 'priceless': 3999, 'palac': 4000, 'nyc': 4001, 'novak': 4002, 'mtv': 4003, 'marion': 4004, 'lampoon': 4005, 'hay': 4006, 'greed': 4007, 'firm': 4008, 'exquisit': 4009, 'errol': 4010, 'enthusiasm': 4011, 'edgi': 4012, 'dolph': 4013, 'disord': 4014, 'alison': 4015, 'alarm': 4016, 'agenda': 4017, '3d': 4018, 'zizek': 4019, 'what': 4020, 'valentin': 4021, 'unleash': 4022, 'thompson': 4023, 'testament': 4024, 'spain': 4025, 'simultan': 4026, 'showdown': 4027, 'session': 4028, 'sergeant': 4029, 'randomli': 4030, 'profit': 4031, 'preserv': 4032, 'ponder': 4033, 'petti': 4034, 'peril': 4035, 'peck': 4036, 'pamela': 4037, 'orlean': 4038, 'nun': 4039, 'minimum': 4040, 'israel': 4041, 'iraq': 4042, 'glanc': 4043, 'gestur': 4044, 'eastern': 4045, 'drip': 4046, 'distort': 4047, 'crown': 4048, 'coup': 4049, 'championship': 4050, 'cassidi': 4051, 'campaign': 4052, 'beatl': 4053, 'angela': 4054, '1933': 4055, '13th': 4056, 'wig': 4057, 'valley': 4058, 'unimagin': 4059, 'travesti': 4060, 'stroke': 4061, 'stake': 4062, 'shootout': 4063, 'scotland': 4064, 'sabrina': 4065, 'rout': 4066, 'restrain': 4067, 'reson': 4068, 'represent': 4069, 'regist': 4070, 'realm': 4071, 'quinn': 4072, 'perpetu': 4073, 'mon': 4074, 'miyazaki': 4075, 'kurosawa': 4076, 'jan': 4077, 'han': 4078, 'gentleman': 4079, 'fido': 4080, 'exposit': 4081, 'empathi': 4082, 'din': 4083, 'crow': 4084, 'cream': 4085, 'crawl': 4086, 'cooki': 4087, 'contradict': 4088, 'climat': 4089, 'calib': 4090, 'buster': 4091, 'bro': 4092, 'brenda': 4093, '1984': 4094, 'wax': 4095, 'warrant': 4096, 'ustinov': 4097, 'unseen': 4098, 'unsatisfi': 4099, 'traumat': 4100, 'tacki': 4101, 'sucker': 4102, 'stargat': 4103, 'spacey': 4104, 'soderbergh': 4105, 'shoddi': 4106, 'shaki': 4107, 'sammi': 4108, 'ross': 4109, 'pretens': 4110, 'pole': 4111, 'perceiv': 4112, 'passabl': 4113, 'painter': 4114, 'monoton': 4115, 'meryl': 4116, 'mclaglen': 4117, 'josh': 4118, 'greg': 4119, 'geek': 4120, 'fuller': 4121, 'femm': 4122, 'distress': 4123, 'derang': 4124, 'demis': 4125, 'delic': 4126, 'darren': 4127, 'dana': 4128, 'crawford': 4129, 'compromis': 4130, 'cloud': 4131, 'censor': 4132, 'businessman': 4133, 'baldwin': 4134, 'absent': 4135, 'abomin': 4136, '1997': 4137, '1987': 4138, 'wholli': 4139, 'verbal': 4140, 'valid': 4141, 'unravel': 4142, 'uncov': 4143, 'tech': 4144, 'tarantino': 4145, 'sid': 4146, 'seal': 4147, 'reluct': 4148, 'primit': 4149, 'polici': 4150, 'norm': 4151, 'nathan': 4152, 'kumar': 4153, 'judgment': 4154, 'jewel': 4155, 'furiou': 4156, 'fog': 4157, 'fenc': 4158, 'expedit': 4159, 'exclus': 4160, 'deniro': 4161, 'dee': 4162, 'deceas': 4163, 'correctli': 4164, 'click': 4165, 'clash': 4166, 'austin': 4167, 'antonioni': 4168, 'anchor': 4169, 'accuraci': 4170, '1993': 4171, 'wretch': 4172, 'wang': 4173, 'vanc': 4174, 'unfair': 4175, 'trait': 4176, 'temper': 4177, 'tax': 4178, 'sustain': 4179, 'sunni': 4180, 'slam': 4181, 'sheet': 4182, 'shanghai': 4183, 'seldom': 4184, 'sand': 4185, 'roller': 4186, 'ritual': 4187, 'pocket': 4188, 'patienc': 4189, 'nicola': 4190, 'murray': 4191, 'mode': 4192, 'malon': 4193, 'logan': 4194, 'joel': 4195, 'hallucin': 4196, 'fought': 4197, 'fart': 4198, 'fabric': 4199, 'enforc': 4200, 'dreck': 4201, 'debt': 4202, 'darn': 4203, 'crippl': 4204, 'conduct': 4205, 'clerk': 4206, 'behold': 4207, 'bake': 4208, 'alec': 4209, '3000': 4210, '2008': 4211, '1995': 4212, '1971': 4213, 'technicolor': 4214, 'tactic': 4215, 'sweep': 4216, 'stuart': 4217, 'stark': 4218, 'squad': 4219, 'soup': 4220, 'shell': 4221, 'scriptwrit': 4222, 'schedul': 4223, 'runner': 4224, 'robber': 4225, 'rita': 4226, 'preston': 4227, 'preposter': 4228, 'phil': 4229, 'pete': 4230, 'penni': 4231, 'outlin': 4232, 'legaci': 4233, 'isabel': 4234, 'helpless': 4235, 'guitar': 4236, 'grief': 4237, 'fundament': 4238, 'exhaust': 4239, 'divid': 4240, 'despis': 4241, 'critiqu': 4242, 'conscious': 4243, 'clau': 4244, 'canyon': 4245, 'bridget': 4246, 'bias': 4247, 'vomit': 4248, 'unexpectedli': 4249, 'sugar': 4250, 'sniper': 4251, 'sentinel': 4252, 'russia': 4253, 'restrict': 4254, 'rehash': 4255, 'rear': 4256, 'propos': 4257, 'passag': 4258, 'palanc': 4259, 'newman': 4260, 'marc': 4261, 'liberti': 4262, 'lacklust': 4263, 'kansa': 4264, 'jodi': 4265, 'jacket': 4266, 'invad': 4267, 'inabl': 4268, 'implic': 4269, 'gregori': 4270, 'flair': 4271, 'drove': 4272, 'downey': 4273, 'delv': 4274, 'culmin': 4275, 'consciou': 4276, 'connor': 4277, 'cigarett': 4278, 'boyl': 4279, 'bloom': 4280, 'alley': 4281, 'alicia': 4282, 'agenc': 4283, 'yeti': 4284, 'wrench': 4285, 'vet': 4286, 'tripe': 4287, 'tendenc': 4288, 'sharon': 4289, 'rod': 4290, 'rehears': 4291, 'rampag': 4292, 'pale': 4293, 'mccoy': 4294, 'lush': 4295, 'ladder': 4296, 'kolchak': 4297, 'karl': 4298, 'improb': 4299, 'horn': 4300, 'foxx': 4301, 'feat': 4302, 'delet': 4303, 'chainsaw': 4304, 'cap': 4305, 'behaviour': 4306, 'bacon': 4307, 'awhil': 4308, 'asylum': 4309, 'arrow': 4310, 'aesthet': 4311, '22': 4312, '1936': 4313, 'wildli': 4314, 'weav': 4315, 'wagner': 4316, 'visitor': 4317, 'underneath': 4318, 'tomorrow': 4319, 'thunderbird': 4320, 'tasteless': 4321, 'suspicion': 4322, 'sung': 4323, 'suffici': 4324, 'stream': 4325, 'spice': 4326, 'shortcom': 4327, 'scoop': 4328, 'rumor': 4329, 'rhythm': 4330, 'prank': 4331, 'paramount': 4332, 'paradis': 4333, 'newcom': 4334, 'minu': 4335, 'lurk': 4336, 'loneli': 4337, 'hungri': 4338, 'hulk': 4339, 'hackney': 4340, 'globe': 4341, 'fright': 4342, 'financ': 4343, 'filler': 4344, 'elit': 4345, 'el': 4346, 'conscienc': 4347, 'coaster': 4348, 'basing': 4349, 'aristocrat': 4350, 'amazon': 4351, '19th': 4352, '1988': 4353, '1978': 4354, '1920': 4355, 'wwe': 4356, 'worship': 4357, 'tierney': 4358, 'teas': 4359, 'straightforward': 4360, 'standout': 4361, 'springer': 4362, 'smell': 4363, 'secondari': 4364, 'rub': 4365, 'recogniz': 4366, 'ram': 4367, 'quietli': 4368, 'posey': 4369, 'penn': 4370, 'paxton': 4371, 'naughti': 4372, 'minist': 4373, 'literari': 4374, 'leigh': 4375, 'lectur': 4376, 'iv': 4377, 'inmat': 4378, 'ingeni': 4379, 'impos': 4380, 'immers': 4381, 'hopkin': 4382, 'heist': 4383, 'grudg': 4384, 'entranc': 4385, 'en': 4386, 'dirt': 4387, 'curli': 4388, 'counterpart': 4389, 'couch': 4390, 'choppi': 4391, 'chavez': 4392, 'chamberlain': 4393, 'cancer': 4394, 'brit': 4395, 'bread': 4396, 'beverli': 4397, 'atroc': 4398, 'abrupt': 4399, '75': 4400, '1989': 4401, '1939': 4402, 'yearn': 4403, 'watcher': 4404, 'variat': 4405, 'transcend': 4406, 'sublim': 4407, 'skeptic': 4408, 'sassi': 4409, 'ratso': 4410, 'quaid': 4411, 'policeman': 4412, 'nolan': 4413, 'net': 4414, 'nemesi': 4415, 'moreov': 4416, 'morbid': 4417, 'missil': 4418, 'misguid': 4419, 'lindsay': 4420, 'laurenc': 4421, 'injuri': 4422, 'heartfelt': 4423, 'geni': 4424, 'esther': 4425, 'entitl': 4426, 'enthral': 4427, 'duel': 4428, 'convert': 4429, 'clan': 4430, 'cattl': 4431, 'bernard': 4432, 'attenborough': 4433, 'ace': 4434, '1986': 4435, 'youngest': 4436, 'vader': 4437, 'unexplain': 4438, 'uncut': 4439, 'tyler': 4440, 'steadi': 4441, 'spiral': 4442, 'setup': 4443, 'rosemari': 4444, 'reliabl': 4445, 'puppi': 4446, 'poe': 4447, 'out': 4448, 'obstacl': 4449, 'mytholog': 4450, 'moder': 4451, 'kitti': 4452, 'kidman': 4453, 'hopelessli': 4454, 'hk': 4455, 'grin': 4456, 'graini': 4457, 'facil': 4458, 'enabl': 4459, 'egg': 4460, 'dont': 4461, 'diari': 4462, 'cruelti': 4463, 'characteris': 4464, 'carlito': 4465, 'bye': 4466, 'buzz': 4467, 'brood': 4468, 'bean': 4469, 'artsi': 4470, '1979': 4471, 'weather': 4472, 'underworld': 4473, 'sweat': 4474, 'spontan': 4475, 'preming': 4476, 'patricia': 4477, 'oblig': 4478, 'niec': 4479, 'narrow': 4480, 'martha': 4481, 'kline': 4482, 'heap': 4483, 'hammi': 4484, 'hain': 4485, 'gina': 4486, 'gillian': 4487, 'fuel': 4488, 'exterior': 4489, 'effici': 4490, 'disastr': 4491, 'despic': 4492, 'decept': 4493, 'clueless': 4494, 'christin': 4495, 'bronson': 4496, 'brendan': 4497, 'bounc': 4498, 'bewar': 4499, 'baffl': 4500, 'athlet': 4501, 'acquaint': 4502, '1969': 4503, 'virtu': 4504, 'viewpoint': 4505, 'uh': 4506, 'trigger': 4507, 'tick': 4508, 'taboo': 4509, 'suprem': 4510, 'sooner': 4511, 'sleepwalk': 4512, 'shatter': 4513, 'scar': 4514, 'rome': 4515, 'renaiss': 4516, 'preachi': 4517, 'outlaw': 4518, 'mermaid': 4519, 'mayhem': 4520, 'loi': 4521, 'loath': 4522, 'lester': 4523, 'insipid': 4524, 'injur': 4525, 'housewif': 4526, 'hepburn': 4527, 'headach': 4528, 'harmless': 4529, 'goof': 4530, 'fontain': 4531, 'enlist': 4532, 'dilemma': 4533, 'dandi': 4534, 'circu': 4535, 'candl': 4536, 'biker': 4537, 'astound': 4538, 'angst': 4539, 'analyz': 4540, '73': 4541, '19': 4542, 'zoom': 4543, 'whore': 4544, 'tripl': 4545, 'surgeri': 4546, 'stimul': 4547, 'steer': 4548, 'stair': 4549, 'spade': 4550, 'sox': 4551, 'slimi': 4552, 'scorses': 4553, 'salt': 4554, 'redund': 4555, 'phenomenon': 4556, 'overlong': 4557, 'oldest': 4558, 'macho': 4559, 'intric': 4560, 'immatur': 4561, 'idol': 4562, 'hostag': 4563, 'hooker': 4564, 'hokey': 4565, 'guin': 4566, 'glorifi': 4567, 'gere': 4568, 'foolish': 4569, 'fluff': 4570, 'filth': 4571, 'ebert': 4572, 'dismal': 4573, 'dish': 4574, 'corbett': 4575, 'contempt': 4576, 'claustrophob': 4577, 'cassavet': 4578, 'camcord': 4579, 'boston': 4580, 'bent': 4581, 'ariel': 4582, 'amor': 4583, 'zane': 4584, 'widescreen': 4585, 'trivia': 4586, 'transplant': 4587, 'strongest': 4588, 'spree': 4589, 'spinal': 4590, 'shred': 4591, 'shield': 4592, 'schlock': 4593, 'rhyme': 4594, 'remad': 4595, 'radiat': 4596, 'proport': 4597, 'preced': 4598, 'perman': 4599, 'obligatori': 4600, 'nolt': 4601, 'naschi': 4602, 'mutual': 4603, 'muscl': 4604, 'mount': 4605, 'messi': 4606, 'margin': 4607, 'keen': 4608, 'joker': 4609, 'harold': 4610, 'gasp': 4611, 'gabl': 4612, 'frantic': 4613, 'flirt': 4614, 'flashi': 4615, 'flag': 4616, 'fascist': 4617, 'faint': 4618, 'dwarf': 4619, 'down': 4620, 'cush': 4621, 'cow': 4622, 'corman': 4623, 'conquer': 4624, 'cohen': 4625, 'beard': 4626, 'astronaut': 4627, 'assert': 4628, 'antagonist': 4629, 'alvin': 4630, '1981': 4631, '1976': 4632, 'www': 4633, 'wield': 4634, 'vaniti': 4635, 'triangl': 4636, 'strive': 4637, 'someday': 4638, 'sensual': 4639, 'scandal': 4640, 'ritchi': 4641, 'resum': 4642, 'repris': 4643, 'raj': 4644, 'persuad': 4645, 'off': 4646, 'neurot': 4647, 'mol': 4648, 'mobil': 4649, 'interestingli': 4650, 'instruct': 4651, 'info': 4652, 'inflict': 4653, 'hara': 4654, 'flock': 4655, 'fishburn': 4656, 'divin': 4657, 'discern': 4658, 'departur': 4659, 'deer': 4660, 'danish': 4661, 'claud': 4662, 'carey': 4663, 'brush': 4664, 'boob': 4665, 'bitten': 4666, 'barn': 4667, 'bachelor': 4668, 'archiv': 4669, 'aborigin': 4670, '95': 4671, '28': 4672, '1945': 4673, 'wendigo': 4674, 'vibrant': 4675, 'undermin': 4676, 'traffic': 4677, 'timberlak': 4678, 'submit': 4679, 'senior': 4680, 'rot': 4681, 'recit': 4682, 'prophet': 4683, 'proclaim': 4684, 'pixar': 4685, 'pickford': 4686, 'parson': 4687, 'pacif': 4688, 'neill': 4689, 'mobster': 4690, 'miracul': 4691, 'melissa': 4692, 'luka': 4693, 'loretta': 4694, 'kathryn': 4695, 'jade': 4696, 'ish': 4697, 'hug': 4698, 'hilar': 4699, 'helm': 4700, 'heartwarm': 4701, 'harrison': 4702, 'hapless': 4703, 'frontier': 4704, 'fragil': 4705, 'europa': 4706, 'earnest': 4707, 'dylan': 4708, 'dim': 4709, 'dame': 4710, 'cycl': 4711, 'colin': 4712, 'cliffhang': 4713, 'clad': 4714, 'cher': 4715, 'cb': 4716, 'casino': 4717, 'carlo': 4718, 'biblic': 4719, 'bate': 4720, 'banter': 4721, 'axe': 4722, 'artwork': 4723, 'anton': 4724, 'winchest': 4725, 'wardrob': 4726, 'vile': 4727, 'venom': 4728, 'vanessa': 4729, 'uma': 4730, 'trier': 4731, 'toronto': 4732, 'token': 4733, 'static': 4734, 'sicken': 4735, 'shepherd': 4736, 'seedi': 4737, 'rooki': 4738, 'redneck': 4739, 'razor': 4740, 'pc': 4741, 'orphan': 4742, 'northern': 4743, 'nope': 4744, 'misfortun': 4745, 'milo': 4746, 'mathieu': 4747, 'mason': 4748, 'marlon': 4749, 'lui': 4750, 'lucil': 4751, 'legitim': 4752, 'jordan': 4753, 'jo': 4754, 'isra': 4755, 'illus': 4756, 'http': 4757, 'holocaust': 4758, 'foil': 4759, 'flavor': 4760, 'feast': 4761, 'estrang': 4762, 'eli': 4763, 'electron': 4764, 'choke': 4765, 'cerebr': 4766, 'breakfast': 4767, 'bondag': 4768, 'blatantli': 4769, 'bikini': 4770, 'articl': 4771, 'aris': 4772, 'antholog': 4773, 'alexandr': 4774, 'akin': 4775, 'wrestler': 4776, 'turd': 4777, 'tack': 4778, 'swept': 4779, 'styliz': 4780, 'smack': 4781, 'shorter': 4782, 'retriev': 4783, 'psych': 4784, 'peer': 4785, 'outdat': 4786, 'oppon': 4787, 'nightclub': 4788, 'magician': 4789, 'linear': 4790, 'leather': 4791, 'knightley': 4792, 'ideolog': 4793, 'huston': 4794, 'howl': 4795, 'highway': 4796, 'gunga': 4797, 'glare': 4798, 'gilbert': 4799, 'frog': 4800, 'fifth': 4801, 'feminin': 4802, 'dudley': 4803, 'disregard': 4804, 'deem': 4805, 'comprehend': 4806, 'clinic': 4807, 'charlton': 4808, 'ceremoni': 4809, 'cartoonish': 4810, 'boyer': 4811, 'audrey': 4812, 'affleck': 4813, 'abund': 4814, 'whack': 4815, 'uniformli': 4816, 'toe': 4817, 'tara': 4818, 'summar': 4819, 'spine': 4820, 'spawn': 4821, 'snatch': 4822, 'sleaz': 4823, 'senat': 4824, 'salman': 4825, 'potter': 4826, 'plate': 4827, 'phenomen': 4828, 'newer': 4829, 'monument': 4830, 'moe': 4831, 'mitch': 4832, 'lighter': 4833, 'lifeless': 4834, 'lavish': 4835, 'greet': 4836, 'goldsworthi': 4837, 'evolut': 4838, 'energet': 4839, 'einstein': 4840, 'durat': 4841, 'deliver': 4842, 'cuban': 4843, 'corn': 4844, 'conrad': 4845, 'compris': 4846, 'collector': 4847, 'client': 4848, 'chip': 4849, 'cemeteri': 4850, 'btw': 4851, 'breakdown': 4852, 'braveheart': 4853, 'boo': 4854, 'bogu': 4855, 'bastard': 4856, '4th': 4857, '1994': 4858, '1991': 4859, 'wtf': 4860, 'undertak': 4861, 'undead': 4862, 'trauma': 4863, 'spectacl': 4864, 'sorrow': 4865, 'signal': 4866, 'replay': 4867, 'randolph': 4868, 'pronounc': 4869, 'outright': 4870, 'ol': 4871, 'occup': 4872, 'nina': 4873, 'neatli': 4874, 'mcqueen': 4875, 'luxuri': 4876, 'liu': 4877, 'lex': 4878, 'kent': 4879, 'kazan': 4880, 'jule': 4881, 'judd': 4882, 'jedi': 4883, 'jare': 4884, 'jam': 4885, 'inaccuraci': 4886, 'ie': 4887, 'historian': 4888, 'healthi': 4889, 'gilliam': 4890, 'fluid': 4891, 'firmli': 4892, 'evelyn': 4893, 'embark': 4894, 'eleven': 4895, 'creek': 4896, 'constitut': 4897, 'clara': 4898, 'cecil': 4899, 'capot': 4900, 'bulk': 4901, 'bori': 4902, 'belli': 4903, 'armstrong': 4904, 'appl': 4905, 'alleg': 4906, '1977': 4907, '1974': 4908, 'walt': 4909, 'vignett': 4910, 'vain': 4911, 'unsuspect': 4912, 'unattract': 4913, 'truman': 4914, 'tokyo': 4915, 'subtli': 4916, 'spray': 4917, 'sidewalk': 4918, 'sacrif': 4919, 'rosario': 4920, 'roar': 4921, 'relentless': 4922, 'propheci': 4923, 'porter': 4924, 'poker': 4925, 'pioneer': 4926, 'pepper': 4927, 'paula': 4928, 'palm': 4929, 'mum': 4930, 'miniseri': 4931, 'miami': 4932, 'meal': 4933, 'lauren': 4934, 'lanc': 4935, 'knee': 4936, 'kiddi': 4937, 'inclus': 4938, 'inaccur': 4939, 'id': 4940, 'groan': 4941, 'goldblum': 4942, 'genet': 4943, 'galaxi': 4944, 'fruit': 4945, 'forgiven': 4946, 'decapit': 4947, 'curtain': 4948, 'congratul': 4949, 'conan': 4950, 'comprehens': 4951, 'comb': 4952, 'carmen': 4953, 'cape': 4954, 'bsg': 4955, 'blur': 4956, 'basket': 4957, 'bait': 4958, 'aussi': 4959, 'ash': 4960, 'antonio': 4961, 'abound': 4962, '1985': 4963, 'weari': 4964, 'weaker': 4965, 'victorian': 4966, 'verg': 4967, 'vastli': 4968, 'turtl': 4969, 'substanti': 4970, 'spill': 4971, 'sparkl': 4972, 'sophi': 4973, 'scariest': 4974, 'scarfac': 4975, 'reincarn': 4976, 'rapidli': 4977, 'profil': 4978, 'playboy': 4979, 'orchestr': 4980, 'optimist': 4981, 'omin': 4982, 'motorcycl': 4983, 'monti': 4984, 'modest': 4985, 'mice': 4986, 'masterson': 4987, 'macabr': 4988, 'jill': 4989, 'ingrid': 4990, 'incorrect': 4991, 'hostil': 4992, 'handicap': 4993, 'hackman': 4994, 'growth': 4995, 'ghetto': 4996, 'frontal': 4997, 'evan': 4998, 'epitom': 4999}

out = predict_fn(test_review, net)
print(out)
print(int(out))


cpu
Inferring sentiment of input data.
1
1


### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [155]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-02-12-07-46-51-409
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-02-12-07-46-51-409


---------------------------------------------------------------!

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [156]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(int(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [157]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [158]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.838

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [159]:
predictor.predict(test_review)

b'1'

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 7 (again): Use the model for the web app

> **TODO:** This entire section and the next contain tasks for you to complete, mostly using the AWS console.

So far we have been accessing our model endpoint by constructing a predictor object which uses the endpoint and then just using the predictor object to perform inference. What if we wanted to create a web app which accessed our model? The way things are set up currently makes that not possible since in order to access a SageMaker endpoint the app would first have to authenticate with AWS using an IAM role which included access to SageMaker endpoints. However, there is an easier way! We just need to use some additional AWS services.

<img src="Web App Diagram.svg">

The diagram above gives an overview of how the various services will work together. On the far right is the model which we trained above and which is deployed using SageMaker. On the far left is our web app that collects a user's movie review, sends it off and expects a positive or negative sentiment in return.

In the middle is where some of the magic happens. We will construct a Lambda function, which you can think of as a straightforward Python function that can be executed whenever a specified event occurs. We will give this function permission to send and recieve data from a SageMaker endpoint.

Lastly, the method we will use to execute the Lambda function is a new endpoint that we will create using API Gateway. This endpoint will be a url that listens for data to be sent to it. Once it gets some data it will pass that data on to the Lambda function and then return whatever the Lambda function returns. Essentially it will act as an interface that lets our web app communicate with the Lambda function.

### Setting up a Lambda function

The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.

#### Part A: Create an IAM Role for the Lambda function

Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.

Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.

In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.

Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**.

#### Part B: Create a Lambda function

Now it is time to actually create the Lambda function.

Using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.

On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. In our example, we will use the code below. 

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below.

In [160]:
predictor.endpoint

'sagemaker-pytorch-2019-02-12-07-46-51-409'

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_api`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 4: Deploying our web app

Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.

In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\*\*REPLACE WITH PUBLIC API URL\*\***. Replace this string with the url that you wrote down in the last step and then save the file.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!

> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill.

**TODO:** Make sure that you include the edited `index.html` file in your project submission.

Now that your web app is working, trying playing around with it and see how well it works.

**Question**: Give an example of a review that you entered into your web app. What was the predicted sentiment of your example review?

**Answer:**
Example Review from RottenTomato
"It's a great start to the season, providing Ali a wonderful character in Wayne Hays, and creator Nic Pizzolatto and crew have crafted a mesmerizing beginning about the power of the past as someone loses grip of it."

It's predicted to be positive. And, that's correct! :)

Another example review from RottenTomato
"There are a few thoughtfully placed cameras and thrilling moments - Bruce Willis vs. a door, for one - but they're not nearly enough to make this self-conscious live-action comic book worthwhile."

It's predicted to be negative. And, that's correct, too! :)

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [161]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: sagemaker-pytorch-2019-02-12-07-46-51-409
